# Library

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime
import math
from numpy import log as ln

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# fold path

In [3]:
drive = 'D'

In [4]:
ods_fold = f'{drive}:/data/big2/ODS'
big2_fold = f'{drive}:/data/big2'
pre_fold = f'{drive}:/data/seoul_lab/pre'
an_fold = f'{drive}:/data/seoul_lab/analysis'
raw1_fold = f'{drive}:/data/big2/BD1/raw'
df1_fold = f'{drive}:/data/big2/BD1/df'
an1_fold = f'{drive}:/data/big2/BD1/an'

# Load
- 등록&제원
- 저감장치 부착이력
- 법정동코드
- KOSIS

## 등록&제원

In [5]:
# 전체 컬럼명
'''
'차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호', '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', 
'배출가스인증번호', '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보', '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', 
'차명', '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량', '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', 
'삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보'
'''

"\n'차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호', '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', \n'배출가스인증번호', '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보', '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', \n'차명', '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량', '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', \n'삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보'\n"

In [6]:
# about 4m 30.7s
name = '[ODS]등록정보&제원정보(2022.12.28)'
file_name = f'{name}.csv'
smig = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False, encoding='cp949', usecols=['차대번호', '본거지법정동코드', '차종_등록정보', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN', '차종유형(소분류)', '연료', '배기량', '최초등록일자', '총중량'])
smig.info()

In [ ]:
smig.columns

Index(['본거지법정동코드', '차종_등록정보', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급',
       '차량말소YN', '차종유형(소분류)', '연료', '총중량', '배기량'],
      dtype='object')

In [ ]:
smig['차종유형(소분류)'].unique()

array(['소형', '경형', '중형', '대형', nan], dtype=object)

In [ ]:
smig1 = smig.rename(columns={'본거지법정동코드':'법정동코드', '차종_등록정보':'차종', '차종유형(소분류)':'차종유형'})
smig1.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량'],
      dtype='object')

In [ ]:
smig1['차량말소YN'].value_counts(dropna=False)

차량말소YN
N    25693532
Y     3840515
Name: count, dtype: int64

### 말소제거

In [ ]:
smig2 = smig1[smig1['차량말소YN'] == 'N'].reset_index(drop=True)
smig2.shape

(25693532, 12)

## 저감장치 부착이력(중복차대제거)

In [ ]:
# about 28.3s
# columns = 저감장치종류, TDMS부착여부, 구조변경일자, 부착업체명, 부착일자, 탈거승인일, 차량번호, 차대번호, 저감장치반납여부, 저감장치구분, 삼원촉매장치반납여부
name = '[ODS]저감장치부착이력(중복차대제거)(2022.12.28)'
file_name = f'{name}.csv'
att = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False, encoding='cp949', usecols=['차대번호', '저감장치구분'])
att.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 809255 entries, 0 to 809254
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   차대번호    809255 non-null  object
 1   저감장치구분  809255 non-null  object
dtypes: object(2)
memory usage: 12.3+ MB


In [ ]:
att['저감장치구분'].unique()

array(['엔진교체', '3종', '1종', '2종', '엔진개조', '1종+SCR', '삼원촉매'], dtype=object)

In [ ]:
att.loc[(att['저감장치구분'] == '1종') | (att['저감장치구분'] == '1종+SCR'), '저감장치부착유무'] = '유'

In [ ]:
att['저감장치부착유무'].value_counts(dropna=False)

저감장치부착유무
NaN    512654
유      296601
Name: count, dtype: int64

## 법정동코드

In [ ]:
# about 5.6s
# columns = '법정동코드', '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역', '대기관리권역YN' 
name = '법정동코드_2022.10.14(수정2023.05.04)'
file_name = f'{name}.xlsx'
code = pd.read_excel(os.path.join(big2_fold, file_name), usecols=['법정동코드', '시도', '시구군'])
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46342 entries, 0 to 46341
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   법정동코드   46342 non-null  int64 
 1   시도      46342 non-null  object
 2   시구군     46319 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


## KOSIS(2021)

In [ ]:
# about 0.2s
name = '용도별_차종별_시군구별_자동차1일평균주행거리(2021)'
file_name = f'{name}.xlsx'
kos = pd.read_excel(os.path.join(ods_fold, file_name))
kos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1230 entries, 0 to 1229
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   시도명     1230 non-null   object 
 1   시군구명    1230 non-null   object 
 2   차종      1230 non-null   object 
 3   전체      1230 non-null   float64
dtypes: float64(1), object(3)
memory usage: 38.6+ KB


# 전처리

## 등급 수정

In [ ]:
smig2['배출가스등급'].value_counts(dropna=False)

배출가스등급
2     10960244
3      8623130
1      2771395
4      1511214
5      1154733
X       288966
T2      247483
T3      136301
T4          37
T5          29
Name: count, dtype: int64

In [ ]:
smig2['배출가스등급'].unique()

array(['5', 'X', 'T5', '4', '3', 'T4', 'T3', '2', '1', 'T2'], dtype=object)

In [ ]:
smig2.loc[smig2['배출가스등급'] == '1', '등급'] = '1'
smig2.loc[(smig2['배출가스등급'] == '2') | (smig2['배출가스등급'] == 'T2'), '등급'] = '2'
smig2.loc[(smig2['배출가스등급'] == '3') | (smig2['배출가스등급'] == 'T3'), '등급'] = '3'
smig2.loc[(smig2['배출가스등급'] == '4') | (smig2['배출가스등급'] == 'T4'), '등급'] = '4'
smig2.loc[(smig2['배출가스등급'] == '5') | (smig2['배출가스등급'] == 'T5'), '등급'] = '5'
smig2.loc[smig2['배출가스등급'] == 'X', '등급'] = 'X'

In [18]:
smig2['등급'].value_counts(dropna=False)

등급
2    11207727
3     8759431
1     2771395
4     1511251
5     1154762
X      288966
Name: count, dtype: int64

## 연료 수정

In [19]:
smig2['연료'].value_counts(dropna=False)

연료
휘발유            12177033
경유              9890990
LPG(액화석유가스)     1904188
휘발유 하이브리드       1095741
전기               368693
NaN              146798
경유 하이브리드          35779
CNG(압축천연가스)       30461
수소                28227
LPG 하이브리드         14352
기타연료                933
CNG 하이브리드           304
LNG(액화천연가스)          27
알코올                   3
태양열                   2
등유                    1
Name: count, dtype: int64

In [20]:
smig2['연료'].unique()

array(['휘발유', '경유', nan, '기타연료', '등유', '알코올', 'LPG(액화석유가스)',
       'CNG(압축천연가스)', '휘발유 하이브리드', 'LPG 하이브리드', '전기', 'LNG(액화천연가스)',
       '경유 하이브리드', 'CNG 하이브리드', '수소', '태양열'], dtype=object)

In [21]:
# about 22.9s
smig2.loc[(smig2['연료'] == '휘발유') | (smig2['연료'] == '휘발유 하이브리드'), 'fuel'] = '휘발유'
smig2.loc[(smig2['연료'] == '경유') | (smig2['연료'] == '경유 하이브리드'), 'fuel'] = '경유'
smig2.loc[(smig2['연료'] == 'LPG(액화석유가스)') | (smig2['연료'] == 'LPG 하이브리드'), 'fuel'] = 'LPG'
smig2.loc[(smig2['연료'] == 'CNG(압축천연가스)') | (smig2['연료'] == 'CNG 하이브리드'), 'fuel'] = 'CNG'
smig2.loc[(smig2['연료'] == 'LNG(액화천연가스)') | (smig2['연료'] == 'LNG 하이브리드'), 'fuel'] = 'LNG'
smig2.loc[smig2['연료'] == '전기', 'fuel'] = '전기'
smig2.loc[smig2['연료'] == '수소', 'fuel'] = '수소'
smig2.loc[smig2['연료'] == '태양열', 'fuel'] = '태양열'
smig2.loc[smig2['연료'] == '알코올', 'fuel'] = '알코올'
smig2.loc[smig2['연료'] == '등유', 'fuel'] = '등유'
smig2.loc[smig2['연료'] == '기타연료', 'fuel'] = '기타연료'
smig2.loc[smig2['연료'].isnull() == True, 'fuel'] = np.nan

In [22]:
smig2['fuel'].value_counts(dropna=False)

fuel
휘발유     13272774
경유       9926769
LPG      1918540
전기        368693
NaN       146798
CNG        30765
수소         28227
기타연료         933
LNG           27
알코올            3
태양열            2
등유             1
Name: count, dtype: int64

In [23]:
smig2.shape

(25693532, 14)

In [24]:
smig2['fuel'].value_counts(dropna=False).sum()

25693532

## 중복 차대번호 제거
- 최초등록일자 최신인 데이터 남기기

In [25]:
# 1,020대 감소
smig2.shape[0], len(smig2['차대번호'].unique())

(25693532, 25692512)

In [26]:
smig2['최초등록일자'].describe()

count   25693532.00
mean    20141515.50
std        61276.12
min     19501231.00
25%     20110228.00
50%     20150817.00
75%     20190408.00
max     20221227.00
Name: 최초등록일자, dtype: float64

In [27]:
smig2.sort_values('최초등록일자', ascending=False)['최초등록일자'].head()

25693531    20221227
25672286    20221227
25672262    20221227
25672261    20221227
25672260    20221227
Name: 최초등록일자, dtype: int64

In [28]:
# 36.4s
smig3 = smig2.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
smig3.shape

(25692512, 14)

## 차령 정보 계산 및 데이터 추가

In [32]:
current_yr = int(datetime.today().strftime("%Y"))
current_yr

2023

In [33]:
smig3.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel'],
      dtype='object')

In [34]:
smig3['차령'] = current_yr - smig3['차량연식']
smig3['차령'].describe()

count   25692512.00
mean           8.55
std            6.21
min            0.00
25%            4.00
50%            7.00
75%           12.00
max          123.00
Name: 차령, dtype: float64

In [35]:
smig3['차령'].isnull().sum()

0

In [36]:
smig3.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령'],
      dtype='object')

## 법정동코드로 지역정보 부착
- 시군구명 앞지역명만 추출

In [37]:
smig3.shape

(25692512, 15)

In [38]:
smig4 = smig3.merge(code, on='법정동코드', how='left')
smig4.shape

(25692512, 17)

In [39]:
# 25.9s
smig4.loc[smig4['시구군'].str.split(' ').str.len() != 1, '시구군'].head()

0     고양시 일산서구
9      창원시 의창구
15     전주시 완산구
18     용인시 기흥구
30     청주시 상당구
Name: 시구군, dtype: object

In [40]:
smig4.loc[smig4['시구군'].str.contains('부산') == True, '시구군'].head()

Series([], Name: 시구군, dtype: object)

In [41]:
# 58.3s
smig4['시구군'] = smig4['시구군'].str.split(' ').str[0]
smig4.loc[smig4['시구군'].str.split(' ').str.len() != 1, '시구군'].head()

24226351    NaN
25154486    NaN
25315223    NaN
25341440    NaN
25345354    NaN
Name: 시구군, dtype: object

In [42]:
smig4['시구군'].head()

0    고양시
1    연수구
2     진구
3    제주시
4    제주시
Name: 시구군, dtype: object

## 1일평균주행거리 정보 추가
- 매칭 컬럼(등록&제원|kosis)
- 1\. 시도, 시구군, 차종 | 시도명, 시군구명, 차종
- 2\. 시도, 차종 | 시도명, 차종
- 3\. 시도, 시구군, 차종(nan) | 시도명, 시군구명, 차종(합계)
    - 차종:nan인 데이터는 kosis의 차종:합계의 일일평균주행거리로 매칭

In [43]:
smig4.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군'],
      dtype='object')

In [44]:
kos.columns

Index(['시도명', '시군구명', '차종', '전체'], dtype='object')

In [45]:
smig4.shape

(25692512, 17)

In [46]:
# about 9.1s
# 시도, 시구군, 차종으로 매칭
smig5 = smig4.merge(kos, left_on=['시도', '시구군', '차종'], right_on=['시도명', '시군구명', '차종'], how='left')
smig5.shape

(25692512, 20)

In [47]:
smig5['전체'].isnull().sum()

341950

In [48]:
# 4.8s
smig5y = smig5.loc[smig5['전체'].isnull() == False]
smig5n = smig5.loc[smig5['전체'].isnull() == True]
smig5y.shape, smig5n.shape

((25350562, 20), (341950, 20))

In [49]:
smig5n.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군', '시도명',
       '시군구명', '전체'],
      dtype='object')

In [50]:
smig5n = smig5n.drop(['시도명', '시군구명', '전체'], axis=1)
smig5n.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군'],
      dtype='object')

In [51]:
smig5n.shape

(341950, 17)

In [52]:
kos.shape

(1230, 4)

In [53]:
kos.loc[kos['시군구명'] == '소계'].shape

(85, 4)

In [54]:
# 시도, 차종으로만 매칭
smig5nm = smig5n.merge(kos.loc[kos['시군구명'] == '소계'].reset_index(drop=True), left_on=['시도', '차종'], right_on=['시도명', '차종'], how='left')
smig5nm.shape

(341950, 20)

In [55]:
smig5nm.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군', '시도명',
       '시군구명', '전체'],
      dtype='object')

In [56]:
smig5nm['전체'].isnull().sum()

14

In [57]:
smig5nm.loc[smig5nm['전체'].isnull() == True, ['시도', '시구군', '차종']]

,시도,시구군,차종
341013,대구광역시,달서구,NaN
341057,대구광역시,남구,NaN
341078,서울특별시,구로구,NaN
341326,대구광역시,서구,NaN
341353,경상북도,경주시,NaN
341367,경상남도,창원시,NaN
341451,대구광역시,달성군,NaN
341452,전라남도,순천시,NaN
341489,경상남도,진주시,NaN
341569,울산광역시,북구,NaN


In [58]:
# 차종이 빈값인 데이터는 kosis의 차종이 합계인 데이터로 매칭
for a, b, c in smig5nm.loc[smig5nm['전체'].isnull() == True, ['시도', '시구군', '차종']].values:
    smig5nm.loc[(smig5nm['전체'].isnull() == True) & (smig5nm['시도'] == a) & ((smig5nm['시구군'] == b) & ((smig5nm['차종'].isnull() == True))), '전체'] = kos.loc[(kos['시도명'] == a) & (kos['시군구명'] == b) & (kos['차종'] == '합계'), '전체'].iloc[0]

In [59]:
kos.loc[(kos['시도명'] == a) & (kos['시군구명'] == b) & (kos['차종'] == '합계'), '전체'].iloc[0]

35.7

In [60]:
smig5nm['전체'].isnull().sum()

0

In [61]:
# 6.4s
smig5 = pd.concat([smig5y, smig5nm], ignore_index=True)
smig5.shape

(25692512, 20)

In [62]:
smig5.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군', '시도명',
       '시군구명', '전체'],
      dtype='object')

In [63]:
# 12.1s
smig5 = smig5.rename(columns={'전체':'일일평균주행거리'})
smig5 = smig5.drop(['시도명', '시군구명'], axis=1)
smig5.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군',
       '일일평균주행거리'],
      dtype='object')

In [64]:
smig5['일일평균주행거리'].isnull().sum()

0

## 저감장치 정보 추가

In [65]:
att.columns

Index(['차대번호', '저감장치구분', '저감장치부착유무'], dtype='object')

In [66]:
smig5.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군',
       '일일평균주행거리'],
      dtype='object')

In [67]:
smig5.shape

(25692512, 18)

In [68]:
# 40.9s
smig6 = smig5.merge(att[['차대번호', '저감장치부착유무']], on='차대번호', how='left')
smig6.shape

(25692512, 19)

In [69]:
smig6['저감장치부착유무'].value_counts(dropna=False)

저감장치부착유무
NaN    25549321
유        143191
Name: count, dtype: int64

## [출력] 전처리 완료

In [70]:
# 2m 47.1s
smig6.to_csv(os.path.join(pre_fold, '전처리완료_일일평균주행거리포함.csv'), index=False)

# [로드] 전처리

In [5]:
# 1m 11.4s
name = '전처리완료_일일평균주행거리포함'
file_name = f'{name}.csv'
smig6 = pd.read_csv(os.path.join(pre_fold, file_name), low_memory=False)
smig6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25692512 entries, 0 to 25692511
Data columns (total 19 columns):
 #   Column    Dtype  
---  ------    -----  
 0   법정동코드     int64  
 1   차종        object 
 2   차대번호      object 
 3   최초등록일자    int64  
 4   차량연식      int64  
 5   배출가스인증번호  object 
 6   배출가스등급    object 
 7   차량말소YN    object 
 8   차종유형      object 
 9   연료        object 
 10  총중량       float64
 11  배기량       float64
 12  등급        object 
 13  fuel      object 
 14  차령        int64  
 15  시도        object 
 16  시구군       object 
 17  일일평균주행거리  float64
 18  저감장치부착유무  object 
dtypes: float64(3), int64(4), object(12)
memory usage: 3.6+ GB


## 연료로 분류

In [6]:
# 10.2s
# 휘발유, 경유, LPG
dgl = smig6.loc[(smig6['fuel'] == '휘발유') | (smig6['fuel'] == '경유') | (smig6['fuel'] == 'LPG')].reset_index(drop=True)
dgl.shape

(25117098, 19)

In [7]:
# 4.6s
# 나머지 : 전기, 수소, 태양열, CNG, LNG, 등유, 알코올, 기타연료, nan
etc = smig6[(smig6['fuel'] != '휘발유') & (smig6['fuel'] != '경유') & (smig6['fuel'] != 'LPG')].reset_index(drop=True)
etc.shape

(575414, 19)

In [8]:
dgl.shape[0] + etc.shape[0]

25692512

# 배출량 계산

## 자동차-엔진 가열(Hot-start) 배출

### 배출량 식
$$E_{i, j} = VKT \times {EF_i \over {1000}} \times DF \times (1 - {R \over {100}})$$

$E_{i, j}$ : 자동차 $j$의 도로주행시 발생한 오염물질 $i$의 배출량($kg/yr$)

$VKT$ : 주행거리($km/yr$)

$EF_i$ : 차종별, 연료별, 연식별, 차속별 배출계수($g/km$)

$DF$ : 열화계수

$R$ : 저감장치 부착 효율(%)

### 주행거리(VKT)
- [현재 설정] "용도별_차종별_시군구별_자동차주행거리" 자료(KOSIS)(2021)의 1일 평균주행거리 * 365 = 주행거리(km/yr)

In [9]:
# 2.6s
df2 = dgl.copy()

In [10]:
df2['VKT'] = df2['일일평균주행거리'] * 365

In [11]:
df2['VKT'].isnull().sum()

0

### DF(열화계수)
- 필요한 정보 : 연료, 차종, 차종유형, 연식, 차령
- 조건 수정(2023.04.20, 최)
    - 특수 소형 = 승용 소형
    - 승합 대형 = 화물 대형

In [12]:
df2['fuel'].unique()

array(['휘발유', 'LPG', '경유'], dtype=object)

In [13]:
DF_col = ['fuel', '차종', '차종유형', '차량연식', '차령']

In [14]:
df2[DF_col].head()

,fuel,차종,차종유형,차량연식,차령
0,휘발유,승용,중형,2023,0
1,휘발유,승용,중형,2023,0
2,휘발유,승용,대형,2023,0
3,휘발유,승용,대형,2023,0
4,휘발유,승용,대형,2023,0


In [15]:
df2[DF_col].values[0]

array(['휘발유', '승용', '중형', 2023, 0], dtype=object)

#### ❗ DF(열화계수) 코드
- 연료, 차종, 차종유형 설정에 해당되지 않는 차량의 열화계수 값 설정
    - 현재 설정값 : np.nan
    - 고려 설정값 : 1

In [16]:
# about 1m 14.5s
DF_CO_list = []
DF_HC_list = []
DF_PM_list = []
DF_NOx_list = []
# '연료', '차종', '차종유형', '연식', '차령'
for fuel, car_type, car_size, car_birth, car_age in tqdm(df2[DF_col].values):    
    if fuel == '경유':
        if (car_type == '승용') or ( (car_type == '특수') and (car_size == '소형') ): # 최이사님 확인(2023.04.20) : "특수 소형" -> "승용 소형" 조건으로 계산
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
        elif car_type == '승합':
            if car_size == '소형' or car_size == '중형':
                warranty = 5
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.05
                    DF_HC = 1 + (car_age - warranty)*0.05
                    DF_PM = 1 + (car_age - warranty)*0.05
                    DF_NOx = 1 + (car_age - warranty)*0.02
                    if DF_CO > 1.5:
                        DF_CO = 1.5
                    if DF_HC > 1.5:
                        DF_HC = 1.5
                    if DF_PM > 1.5:
                        DF_PM = 1.5
                    if DF_NOx > 1.2:
                        DF_NOx = 1.2
            elif car_size == '대형':
                warranty = 3
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.05
                    DF_HC = 1 + (car_age - warranty)*0.05
                    DF_PM = 1 + (car_age - warranty)*0.05
                    DF_NOx = 1 + (car_age - warranty)*0.02
                    if DF_CO > 1.5:
                        DF_CO = 1.5
                    if DF_HC > 1.5:
                        DF_HC = 1.5
                    if DF_PM > 1.5:
                        DF_PM = 1.5
                    if DF_NOx > 1.2:
                        DF_NOx = 1.2
        elif car_type == '화물':
            # car_size : 경, 소, 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
        elif car_type == '특수':
            # car_size : 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
    elif (fuel == '휘발유') or (fuel == 'LPG'):
        if car_type == '승용':
            # 경, 소, 중, 대
            if car_birth <= 2000:
                warranty = 5
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.1
                    DF_HC = 1 + (car_age - warranty)*0.1
                    DF_PM = 1 + (car_age - warranty)*0.1
                    DF_NOx = 1 + (car_age - warranty)*0.1
                    if DF_CO > 2.0:
                        DF_CO = 2.0
                    if DF_HC > 2.0:
                        DF_HC = 2.0
                    if DF_PM > 2.0:
                        DF_PM = 2.0
                    if DF_NOx > 2.0:
                        DF_NOx = 2.0
            elif car_birth >= 2001:
                warranty = 10
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.1
                    DF_HC = 1 + (car_age - warranty)*0.1
                    DF_PM = 1 + (car_age - warranty)*0.1
                    DF_NOx = 1 + (car_age - warranty)*0.1
                    if DF_CO > 2.0:
                        DF_CO = 2.0
                    if DF_HC > 2.0:
                        DF_HC = 2.0
                    if DF_PM > 2.0:
                        DF_PM = 2.0
                    if DF_NOx > 2.0:
                        DF_NOx = 2.0
        elif car_type == '승합':
            if car_size == '경형':
                warranty = 5
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.1
                    DF_HC = 1 + (car_age - warranty)*0.1
                    DF_PM = 1 + (car_age - warranty)*0.1
                    DF_NOx = 1 + (car_age - warranty)*0.1
                    if DF_CO > 2.0:
                        DF_CO = 2.0
                    if DF_HC > 2.0:
                        DF_HC = 2.0
                    if DF_PM > 2.0:
                        DF_PM = 2.0
                    if DF_NOx > 2.0:
                        DF_NOx = 2.0
            elif (car_size == '소형') or (car_size == '중형'):
                if car_birth <= 2005:
                    warranty = 5
                    if car_age <= warranty:
                        DF_CO = 1
                        DF_HC = 1
                        DF_PM = 1
                        DF_NOx = 1
                    else:
                        DF_CO = 1 + (car_age - warranty)*0.1
                        DF_HC = 1 + (car_age - warranty)*0.1
                        DF_PM = 1 + (car_age - warranty)*0.1
                        DF_NOx = 1 + (car_age - warranty)*0.1
                        if DF_CO > 2.0:
                            DF_CO = 2.0
                        if DF_HC > 2.0:
                            DF_HC = 2.0
                        if DF_PM > 2.0:
                            DF_PM = 2.0
                        if DF_NOx > 2.0:
                            DF_NOx = 2.0
                elif car_birth >= 2006:
                    warranty = 10
                    if car_age <= warranty:
                        DF_CO = 1
                        DF_HC = 1
                        DF_PM = 1
                        DF_NOx = 1
                    else:
                        DF_CO = 1 + (car_age - warranty)*0.1
                        DF_HC = 1 + (car_age - warranty)*0.1
                        DF_PM = 1 + (car_age - warranty)*0.1
                        DF_NOx = 1 + (car_age - warranty)*0.1
                        if DF_CO > 2.0:
                            DF_CO = 2.0
                        if DF_HC > 2.0:
                            DF_HC = 2.0
                        if DF_PM > 2.0:
                            DF_PM = 2.0
                        if DF_NOx > 2.0:
                            DF_NOx = 2.0
        elif (car_type == '화물') or ( (car_type == '승합') and (car_size == '대형')): # 최이사님 확인(2023.04.20) : 습합 대형 -> 화물 대형 조건으로 계산
            # 경, 소, 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.1
                DF_HC = 1 + (car_age - warranty)*0.1
                DF_PM = 1 + (car_age - warranty)*0.1
                DF_NOx = 1 + (car_age - warranty)*0.1
                if DF_CO > 2.0:
                    DF_CO = 2.0
                if DF_HC > 2.0:
                    DF_HC = 2.0
                if DF_PM > 2.0:
                    DF_PM = 2.0
                if DF_NOx > 2.0:
                    DF_NOx = 2.0
        elif car_type == '특수':
            # 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.1
                DF_HC = 1 + (car_age - warranty)*0.1
                DF_PM = 1 + (car_age - warranty)*0.1
                DF_NOx = 1 + (car_age - warranty)*0.1
                if DF_CO > 2.0:
                    DF_CO = 2.0
                if DF_HC > 2.0:
                    DF_HC = 2.0
                if DF_PM > 2.0:
                    DF_PM = 2.0
                if DF_NOx > 2.0:
                    DF_NOx = 2.0
    else:
        # 설정 고민
            # 고려 : 1
        # DF_CO, DF_HC, DF_PM, DF_NOx = np.nan, np.nan, np.nan, np.nan
        DF_CO, DF_HC, DF_PM, DF_NOx = 1, 1, 1, 1

    # 열화계수(DF) 리스트에 저장
    DF_CO_list.append(DF_CO)
    DF_HC_list.append(DF_HC)
    DF_PM_list.append(DF_PM)
    DF_NOx_list.append(DF_NOx)

100%|██████████| 25117098/25117098 [01:07<00:00, 370605.24it/s]


In [17]:
df2.shape[0]

25117098

In [18]:
len(DF_CO_list), len(DF_HC_list), len(DF_PM_list), len(DF_NOx_list)

(25117098, 25117098, 25117098, 25117098)

In [19]:
df2['DF_CO'] = DF_CO_list
df2['DF_HC'] = DF_HC_list
df2['DF_PM'] = DF_PM_list
df2['DF_NOx'] = DF_NOx_list

In [20]:
check_DF_col = ['DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx']
df2.loc[df2['차령'] < 10, DF_col + check_DF_col].head()

,fuel,차종,차종유형,차량연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx
0,휘발유,승용,중형,2023,0,1.00,1.00,1.00,1.00
1,휘발유,승용,중형,2023,0,1.00,1.00,1.00,1.00
2,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00
3,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00
4,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00


### 저감장치 부착 효율(R) 계산
- 저감장치부착유무 : 유 -> DPF로 가정하고 진행
- 휘발성 유기 화합물(Volatile Organic Compounds:VOC) : 생활주변에서 흔히 사용하는 탄화수소류가 거의 해당됨.
- 저감장치별 물질 제거 효율(%)

    장치종류|CO|VOC|PM
    :-:|:-:|:-:|:-:
    DPF|99.5|90|83.6
    pDPF|94.6|89.3|56
    DOC|85.4|72|35

In [21]:
df2.loc[df2['저감장치부착유무'] == '유', ['R_CO', 'R_HC', 'R_PM']] = 99.5, 90, 83.6

In [22]:
df2.shape[0]

25117098

In [23]:
check_R_col = ['R_CO', 'R_HC', 'R_PM']

In [24]:
df2[check_R_col].notnull().sum()

R_CO    143184
R_HC    143184
R_PM    143184
dtype: int64

In [25]:
df2.loc[df2['저감장치부착유무'] == '유', DF_col + check_DF_col + check_R_col].head()

,fuel,차종,차종유형,차량연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx,R_CO,R_HC,R_PM
27396,경유,승용,중형,2005,18,1.50,1.50,1.50,1.20,99.50,90.00,83.60
33079,경유,승용,중형,2005,18,1.50,1.50,1.50,1.20,99.50,90.00,83.60
82122,경유,승용,대형,2003,20,1.50,1.50,1.50,1.20,99.50,90.00,83.60
99157,경유,승합,중형,2007,16,1.50,1.50,1.50,1.20,99.50,90.00,83.60
132058,휘발유,승용,중형,2006,17,1.70,1.70,1.70,1.70,99.50,90.00,83.60


In [26]:
df2[check_R_col] = df2[check_R_col].fillna(0)

In [27]:
df2[check_R_col].isnull().sum()

R_CO    0
R_HC    0
R_PM    0
dtype: int64

### ❗ 배출계수(EFi)
- 연료, 차종, 차종유형 설정에 해당되지 않는 차량의 배출계수 값 설정
    - 현재 설정값 : np.nan
    - 고려 설정값 : 1
- 조건 수정(2023.04.20, 최)
    - 특수 중형 = 승합 중형
    - 화물 중형 = 승합 중형
    - 화물 대형 = 승합 대형
    - 특수 중형 = 승용 중형

In [28]:
EFi_col = ['차종', '차종유형', 'fuel', '차량연식']
# 참고 : KOSIS 차량속도(2017) 일반국도 평균 (https://kosis.kr/statHtml/statHtml.do?orgId=210&tblId=DT_21002_J008)
V = 54.1
# 국가 대기오염물질 배출량 산정방법 편람(V)(2022) 부록 참고(for PM-2.5)
k = 0.92

In [29]:
df2[EFi_col].values[0]

array(['승용', '중형', '휘발유', 2023], dtype=object)

In [30]:
df2['차종유형'].unique()

array(['중형', '대형', '경형', '소형', nan], dtype=object)

In [31]:
# about 5m 40s
EFi_CO_list = []
EFi_HC_list = []
EFi_NOx_list = []
EFi_PM10_list = []
EFi_PM2_5_list = []
EFi_NH3_list = []

# 차종, 차종유형, 연료, 연식
for car_type, car_size, fuel, car_birth in tqdm(df2[EFi_col].values):
    EFi_COm = 0
    EFi_HCm = 0
    EFi_NOxm = 0
    EFi_PM10m = 0
    EFi_PM2_5m = 0
    EFi_NH3m = 0
    for V in [30, 75, 100]:
        if fuel == '휘발유':
            if (car_type == '승용') and (car_size == '경형'):
                if car_birth <= 1996:
                    if V <= 65:
                        EFi_CO = 59.783 * (V**-1.0007)
                    else:
                        EFi_CO = 0.0874 * V - 3.5618
                    EFi_HC = 7.6244 * (V**-0.8364)
                    EFi_NOx = 2.6754 * (V**-0.3236)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1997 <= car_birth <= 1999:
                    if V <= 65:
                        EFi_CO = 59.783 * (V**-1.0007)
                    else:
                        EFi_CO = 0.0874 * V - 3.5618
                    EFi_HC = 8.6275 * (V**-1.0722)
                    EFi_NOx = 3.2294 * (V**-0.5763)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 2000 <= car_birth <= 2002:
                    if V <= 65:
                        EFi_CO = 60.556 * (V**-1.2501)
                    else:
                        EFi_CO = -0.0006 * V + 0.5753
                    EFi_HC = 5.1835 * (V**-1.1889)
                    EFi_NOx = 1.7525 * (V**-0.6481)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V > 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    if V <= 65:
                        EFi_CO = 60.556 * (V**-1.2501)
                    else:
                        EFi_CO = -0.0006 * V + 0.5753
                    EFi_HC = 0.7446 * (V**-0.9392)
                    EFi_NOx = 0.3403 * (V**-0.5455)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    if V <= 45:
                        EFi_CO = 4.9952 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0001 * V**2 + 0.0229 * V - 0.5701
                    EFi_HC = 0.2958 * (V**-0.7830)
                    EFi_NOx = 0.4819 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth <= 2011:
                    if V <= 45:
                        EFi_CO = 4.5956 * (V**-0.8461)
                    else:
                        EFi_CO = -9.2000*(10**-5) * (V**2) + 2.1068*(10**-2) * V - 5.2449*(10**-1)
                    EFi_HC = 0.2662 * (V**-0.7830)
                    EFi_NOx = 0.4476 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2012 <= car_birth <= 2013:
                    if V <= 45:
                        EFi_CO = 4.4517 * (V**-0.8461)
                    else:
                        EFi_CO = -8.9120*(10**-5) * (V**2) + 2.0408*(10**-2)*V - 5.0807*(10**-1)
                    EFi_HC = 0.2556 * (V**-0.7830)
                    EFi_NOx = 0.4353 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif car_birth == 2014:
                    if V <= 45:
                        EFi_CO = 4.3079 * (V**-0.8461)
                    else:
                        EFi_CO = -8.6240*(10**-5) * (V**2) + 1.9749*(10**-2)*V - 4.9165*(10**-1)
                    EFi_HC = 0.2449 * (V**-0.7830)
                    EFi_NOx = 0.4230 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2015 <= car_birth:
                    if V <= 45:
                        EFi_CO = 4.164 * (V**-0.8461)
                    else:
                        EFi_CO = -8.3360*(10**-5) * (V**2) + 1.9089*(10**-2)*V - 4.7524*(10**-1)
                    EFi_HC = 0.2343 * (V**-0.7830)
                    EFi_NOx = 0.4106 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
            elif (car_type == '승용') and (car_size == '소형'):
                if car_birth <= 1986:
                    EFi_CO = 247.00 * (V**-0.6651)
                    EFi_HC = 15.953 * (V**-0.5059)
                    EFi_NOx = 3.1140 * (V**-0.2278)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1987 <= car_birth <= 1990:
                    EFi_CO = 36.169 * (V**-0.7587)
                    EFi_HC = 15.607 * (V**-1.0423)
                    EFi_NOx = 6.2007 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1991 <= car_birth <= 1999:
                    EFi_CO = 111.67 * (V**-1.1566)
                    EFi_HC = 32.017 * (V**-1.4171)
                    EFi_NOx = 7.5244 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 2000 <= car_birth <= 2002:
                    EFi_CO = 22.356 * (V**-0.9068)
                    EFi_HC = 0.8428 * (V**-0.8829)
                    EFi_NOx = 1.2613 * (V**-0.3873)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V > 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    EFi_CO = 1.4898 * (V**-0.3837)
                    EFi_HC = 0.1738 * (V**-0.7268)
                    EFi_NOx = 0.1563 * (V**-0.2671)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    EFi_CO = 1.0000*(10**-4)*(V**2) - 7.1000*(10**-3)*V + 2.2450*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0633 * (V**-1.0484)
                    else:
                        EFi_HC = 1.3200*(10**-6)*(V**2) - 1.8800*(10**-4)*V + 7.7000*(10**-3)
                    EFi_NOx = -3.5000*(10**-6)*(V**2) + 3.3000*(10**-4)*V + 1.1200*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth <= 2011:
                    EFi_CO = 9.2000*(10**-5)*(V**2) - 6.5320*(10**-3)*V + 2.0654*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0570 * (V**-1.0484)
                    else:
                        EFi_HC = 1.1880*(10**-6)*(V**2) - 1.6920*(10**-4)*V + 6.9300*(10**-3)
                    EFi_NOx = -3.2511*(10**-6)*(V**2) + 3.0653*(10**-4)*V + 1.0404*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2012 <= car_birth <= 2013:
                    EFi_CO = 8.9120*(10**-5)*(V**2) - 6.3275*(10**-3)*V + 2.0007*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0547 * (V**-1.0484)
                    else:
                        EFi_HC = 1.1405*(10**-6)*(V**2) - 1.6243*(10**-4)*V + 6.6528*(10**-3)
                    EFi_NOx = -3.1615*(10**-6)*(V**2) + 2.9809*(10**-4)*V + 1.0117*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif car_birth == 2014:
                    EFi_CO = 8.6240*(10**-5)*(V**2) - 6.1230*(10**-3)*V + 1.9361*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0524 * (V**-1.0484)
                    else:
                        EFi_HC = 1.0930*(10**-6)*(V**2) - 1.5566*(10**-4)*V + 6.3756*(10**-3)
                    EFi_NOx = -3.0719*(10**-6)*(V**2) + 2.8964*(10**-4)*V + 9.8301*(10**-3)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2015 <= car_birth:
                    EFi_CO = 8.3360*(10**-5)*(V**2) - 5.9186*(10**-3)*V + 1.8714*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0501 * (V**-1.0484)
                    else:
                        EFi_HC = 1.0500*(10**-6)*(V**2) - 1.4890*(10**-4)*V + 6.09840*(10**-3)
                    EFi_NOx = -2.9823*(10**-6)*(V**2) + 2.8119*(10**-4)*V + 9.5434*(10**-3)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
            elif ( (car_type == '승용') and (car_size == '중형') ) or ( (car_type == '특수') and (car_size == '중형') ): # 최 확인(2023.04.20) : 특수 중형 -> 승용 중형 조건으로 계산
                if car_birth <= 1986:
                    EFi_CO = 247.00 * (V**-0.6651)
                    EFi_HC = 15.953 * (V**-0.5059)
                    EFi_NOx = 3.1140 * (V**-0.2278)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1987 <= car_birth <= 1990:
                    EFi_CO = 36.169 * (V**-0.7587)
                    EFi_HC = 15.607 * (V**-1.0423)
                    EFi_NOx = 6.2007 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1991 <= car_birth <= 1999:
                    EFi_CO = 51.555 * (V**-0.9531)
                    EFi_HC = 31.816 * (V**-1.4804)
                    EFi_NOx = 7.5244 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 2000 <= car_birth <= 2002:
                    EFi_CO = 29.921 * (V**-0.8868)
                    EFi_HC = 7.9374 * (V**-1.3041)
                    EFi_NOx = 1.8525 * (V**-0.4192)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V > 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    EFi_CO = 2.4938 * (V**-0.6106)
                    EFi_HC = 0.4262 * (V**-1.0122)
                    EFi_NOx = 0.1818 * (V**-0.4316)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    EFi_CO = 2.2900*(10**-5)*(V**2) - 1.6300*(10**-3)*V + 5.8300*(10**-2)
                    if V <= 65.4:
                        EFi_HC = 0.0633 * (V**-1.0484)
                    else:
                        EFi_HC = 1.3200*(10**-6)*(V**2) - 1.8800*(10**-4)*V + 7.7000*(10**-3)
                    EFi_NOx = -3.5000*(10**-6)*(V**2) + 3.3000*(10**-4)*V + 1.1200*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth <= 2011:
                    EFi_CO = 2.1068*(10**-5)*(V**2) - 1.4996*(10**-3)*V + 5.3636*(10**-2)
                    if V <= 65.4:
                        EFi_HC = 0.0570 * (V**-1.0484)
                    else:
                        EFi_HC = 1.1880*(10**-6)*(V**2) - 1.6920*(10**-4)*V + 6.9300*(10**-3)
                    EFi_NOx = -3.2511*(10**-6)*(V**2) + 3.0653*(10**-4)*V + 1.0404*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2012 <= car_birth <= 2013:
                    EFi_CO = 2.0408*(10**-5)*(V**2) - 1.4527*(10**-3)*V + 5.1957*(10**-2)
                    if V <= 65.4:
                        EFi_HC = 0.0547 * (V**-1.0484)
                    else:
                        EFi_HC = 1.1405*(10**-6)*(V**2) - 1.6243*(10**-4)*V + 6.6528*(10**-3)
                    EFi_NOx = -3.1615*(10**-6)*(V**2) + 2.9809*(10**-4)*V + 1.0117*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif car_birth == 2014:
                    EFi_CO = 1.9749*(10**-5)*(V**2) - 1.4057*(10**-3)*V + 5.0278*(10**-2)
                    if V <= 65.4:
                        EFi_HC = 0.0524 * (V**-1.0484)
                    else:
                        EFi_HC = 1.0930*(10**-6)*(V**2) - 1.5566*(10**-4)*V + 6.3756*(10**-3)
                    EFi_NOx = -3.0719*(10**-6)*(V**2) + 2.8964*(10**-4)*V + 9.8301*(10**-3)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2015 <= car_birth:
                    EFi_CO = 1.9089*(10**-5)*(V**2) - 1.3588*(10**-3)*V + 4.8599*(10**-2)
                    if V <= 65.4:
                        EFi_HC = 0.0501 * (V**-1.0484)
                    else:
                        EFi_HC = 1.0500*(10**-6)*(V**2) - 1.4890*(10**-4)*V + 6.0984*(10**-3)
                    EFi_NOx = -2.9823*(10**-6)*(V**2) + 2.8119*(10**-4)*V + 9.5434*(10**-3)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
            elif (car_type == '승용') and (car_size == '대형'):
                if car_birth <= 1986:
                    EFi_CO = 247.00 * (V**-0.6651)
                    EFi_HC = 15.953 * (V**-0.5059)
                    EFi_NOx = 3.1140 * (V**-0.2278)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1987 <= car_birth <= 1990:
                    EFi_CO = 36.169 * (V**-0.7587)
                    EFi_HC = 15.607 * (V**-1.0423)
                    EFi_NOx = 6.2007 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1991 <= car_birth <= 1999:
                    EFi_CO = 51.555 * (V**-0.9531)
                    EFi_HC = 31.816 * (V**-1.4804)
                    EFi_NOx = 7.5244 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 2000 <= car_birth <= 2002:
                    EFi_CO = 29.921 * (V**-0.8868)
                    EFi_HC = 7.9374 * (V**-1.3041)
                    EFi_NOx = 1.8525 * (V**-0.4192)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V > 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    EFi_CO = 2.4938 * (V**-0.6106)
                    EFi_HC = 0.4262 * (V**-1.0122)
                    EFi_NOx = 0.1818 * (V**-0.4316)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    if V <= 65.4:
                        EFi_CO = 1.4082 * (V**-0.7728)
                    else:
                        EFi_CO = 8.0000*(10**-5)*(V**2) - 1.2700*(10**-2)*V + 5.7510*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0633 * (V**-1.0484)
                    else:
                        EFi_HC = 1.3200*(10**-6)*(V**2) - 1.8800*(10**-4)*V + 7.7000*(10**-3)
                    EFi_NOx = -3.5000*(10**-6)*(V**2) + 3.3000*(10**-4)*V + 1.1200*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth <= 2011:
                    if V <= 65.4:
                        EFi_CO = 1.2955 * (V**-0.7728)
                    else:
                        EFi_CO = 7.3600*(10**-5)*(V**2) - 1.1684*(10**-2)*V + 5.2909*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0570 * (V**-1.0484)
                    else:
                        EFi_HC = 1.1880*(10**-6)*(V**2) - 1.6920*(10**-4)*V + 6.9300*(10**-3)
                    EFi_NOx = -3.2511*(10**-6)*(V**2) + 3.0653*(10**-4)*V + 1.0404*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2012 <= car_birth <= 2013:
                    if V <= 65.4:
                        EFi_CO = 1.2550 * (V**-0.7728)
                    else:
                        EFi_CO = 7.1296*(10**-5)*(V**2) - 1.1318*(10**-2)*V + 5.1253*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0547 * (V**-1.0484)
                    else:
                        EFi_HC = 1.1405*(10**-6)*(V**2) - 1.6243*(10**-4)*V + 6.6528*(10**-3)
                    EFi_NOx = -3.1615*(10**-6)*(V**2) + 2.9809*(10**-4)*V + 1.0117*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif car_birth == 2014:
                    if V <= 65.4:
                        EFi_CO = 1.2144 * (V**-0.7728)
                    else:
                        EFi_CO = 6.8992*(10**-5)*(V**2) - 1.0952*(10**-2)*V + 4.9597*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0524 * (V**-1.0484)
                    else:
                        EFi_HC = 1.0930*(10**-6)*(V**2) - 1.5566*(10**-4)*V + 6.3756*(10**-3)
                    EFi_NOx = -3.0719*(10**-6)*(V**2) + 2.8964*(10**-4)*V + 9.8301*(10**-3)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2015 <= car_birth:
                    if V <= 65.4:
                        EFi_CO = 1.1739 * (V**-0.7728)
                    else:
                        EFi_CO = 6.6688*(10**-5)*(V**2) - 1.0587*(10**-2)*V + 4.7940*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0501 * (V**-1.0484)
                    else:
                        EFi_HC = 1.0500*(10**-6)*(V**2) - 1.4890*(10**-4)*V + 6.0984*(10**-3)
                    EFi_NOx = -2.9823*(10**-6)*(V**2) + 2.8119*(10**-4)*V + 9.5434*(10**-3)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
            elif ( (car_type == '승합') and (car_size == '경형') ) or ( (car_type == '화물') and (car_size == '경형') ):
                if car_birth <= 1996:
                    if V <= 45:
                        EFi_CO = 11.249 * (V**-0.6579)
                    else:
                        EFi_CO = 0.0003 * (V**2) + 0.0002 * V + 0.4136
                    EFi_HC = 7.6244 * (V**-0.8364)
                    EFi_NOx = 2.6754 * (V**-0.3236)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1997 <= car_birth <= 1999:
                    if V <= 45:
                        EFi_CO = 16.965 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0003 * (V**2) + 0.0777 * V - 1.9363
                    EFi_HC = 3.0285 * (V**-0.7830)
                    EFi_NOx = 1.9923 * (V**-0.3889)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 2000 <= car_birth <= 2002:
                    if V <= 45:
                        EFi_CO = 9.9433 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0002 * (V**2) + 0.0455 * V - 1.1349
                    EFi_HC = 1.8928 * (V**-0.7830)
                    EFi_NOx = 1.2352 * (V**-0.3889)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V >= 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    if V <= 45:
                        EFi_CO = 9.9433 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0002 * (V**2) + 0.0455 * V - 1.1349
                    EFi_HC = 0.9227 * (V**-0.7830)
                    EFi_NOx = 3.8859 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V >= 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    if V <= 45:
                        EFi_CO = 4.4952 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0001 * (V**2) + 0.0229 * V - 0.5701
                    EFi_HC = 0.2958 * (V**-0.7830)
                    EFi_NOx = 0.4819 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V >= 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth:
                    if V <= 45:
                        EFi_CO = 4.4952 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0001 * (V**2) + 0.0229 * V - 0.5701
                    EFi_HC = 0.2958 * (V**-0.7830)
                    EFi_NOx = 0.4819 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V >= 90:
                        EFi_NH3 = 0.022
            elif (car_type == '승합') and (car_size == '소형'):
                if car_birth <= 1990:
                    EFi_CO = 36.169 * (V**-0.7587)
                    EFi_HC = 15.607 * (V**-1.0423)
                    EFi_NOx = 6.2007 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif 1991 <= car_birth <= 1996:
                    EFi_CO = 39.402 * (V**-0.8879)
                    EFi_HC = 23.400 * (V**-1.4041)
                    EFi_NOx = 7.5244 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif 1997 <= car_birth <= 1999:
                    EFi_CO = 77.088 * (V**-1.2078)
                    EFi_HC = 18.731 * (V**-1.5356)
                    EFi_NOx = 4.4260 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif car_birth == 2000:
                    EFi_CO = 41.669 * (V**-1.2078)
                    EFi_HC = 14.190 * (V**-1.5356)
                    EFi_NOx = 3.4578 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V >= 90:
                        EFi_NH3 = 0.084
                elif 2001 <= car_birth <= 2002:
                    EFi_CO = 41.669 * (V**-1.2078)
                    EFi_HC = 11.920 * (V**-1.5356)
                    EFi_NOx = 3.0649 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V >= 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    EFi_CO = 37.573 * (V**-1.2078)
                    EFi_HC = 3.1786 * (V**-1.5356)
                    EFi_NOx = 1.4931 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V >= 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    EFi_CO = 32.899 * (V**-1.2078)
                    EFi_HC = 2.7387 * (V**-1.5356)
                    EFi_NOx = 1.1808 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V >= 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth:
                    EFi_CO = 32.899 * (V**-1.2078)
                    EFi_HC = 2.7387 * (V**-1.5356)
                    EFi_NOx = 1.1808 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.08
                    elif V >= 90:
                        EFi_NH3 = 0.022
            elif ( (car_type == '승합') and (car_size == '중형') or (car_size == '대형') ) or ( (car_type == '화물') and (car_size == '소형') ):
                if car_birth <= 1990:
                    EFi_CO = 36.169 * (V**-0.7587)
                    EFi_HC = 15.607 * (V**-1.0423)
                    EFi_NOx = 6.2007 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif 1991 <= car_birth <= 1996:
                    EFi_CO = 39.402 * (V**-0.8879)
                    EFi_HC = 23.400 * (V**-1.4041)
                    EFi_NOx = 7.5244 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif 1997 <= car_birth <= 1999:
                    EFi_CO = 77.088 * (V**-1.2078)
                    EFi_HC = 18.731 * (V**-1.5356)
                    EFi_NOx = 4.4260 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif 2000 <= car_birth <= 2002:
                    EFi_CO = 41.669 * (V**-1.2078)
                    EFi_HC = 14.190 * (V**-1.5356)
                    EFi_NOx = 3.4578 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V >= 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    EFi_CO = 36.578 * (V**-1.2078)
                    EFi_HC = 3.0337 * (V**-1.5356)
                    EFi_NOx = 2.0104 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V >= 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    EFi_CO = 14.202 * (V**-1.2078)
                    EFi_HC = 1.2233 * (V**-1.5356)
                    EFi_NOx = 0.2493 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V >= 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth:
                    EFi_CO = 14.202 * (V**-1.2078)
                    EFi_HC = 1.2233 * (V**-1.5356)
                    EFi_NOx = 0.2493 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V >= 90:
                        EFi_NH3 = 0.022
            elif (car_type == '화물') and (car_size == '중형'):
                if 1993 <= car_birth <= 1998:
                    if V <= 65:
                        EFi_CO = 70
                        EFi_HC = 7
                        EFi_NOx = 4.5
                    else:
                        EFi_CO = 55
                        EFi_HC = 3.5
                        EFi_NOx = 7.5
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif car_birth == 1999:
                    if V <= 65:
                        EFi_CO = 70
                        EFi_HC = 7
                        EFi_NOx = 4.5
                    else:
                        EFi_CO = 55
                        EFi_HC = 3.5
                        EFi_NOx = 7.5
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif 2000 <= car_birth <= 2002:
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V > 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2008:
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth:
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
            elif (car_type == '화물') and (car_size == '대형'):
                if 1993 <= car_birth:
                    if V <= 65:
                        EFi_CO = 70
                        EFi_HC = 7
                        EFi_NOx = 4.5
                    else:
                        EFi_CO = 55
                        EFi_HC = 3.5
                        EFi_NOx = 7.5
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                else:
                    EFi_CO = 1
                    EFi_HC = 1
                    EFi_NOx = 1
                    EFi_PM10 = 1
                    EFi_PM2_5 = 1
                EFi_NH3 = 0.002
        elif fuel == '경유':
            if (car_type == '승용') and (car_size == '경형'):
                if car_birth <= 2005:
                    EFi_CO = 0.7392 * (V**-0.7524)
                    EFi_HC = 0.0989 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0839 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0839 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2006 <= car_birth <= 2010:
                    EFi_CO = 0.5775 * (V**-0.7524)
                    EFi_HC = 0.0825 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0420 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0420 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2011 <= car_birth <= 2015:
                    EFi_CO = 0.5141 * (V**-0.6792)
                    EFi_HC = 0.3713 * (V**-0.7513)
                    EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**-0.0416)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416) # EFi_PM2_5 = k * 0.0009 * (V**0.0416)
                    EFi_NH3 = 0.0019
                elif 2016 <= car_birth:
                    EFi_CO = 0.4574 * (V**-0.5215)
                    EFi_HC = 0.1300 * (V**-0.7265)
                    EFi_NOx = 2.7702 * (V**-0.3869)
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**-0.0416)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416) # EFi_PM2_5 = k * 0.0009 * (V**0.0416) 
                    EFi_NH3 = 0.0019
            elif (car_type == '승용') and (car_size == '소형'):
                if car_birth <= 2004:
                    EFi_CO = 5.9672 * (V**-0.9534)
                    EFi_HC = 0.6523 * (V**-1.0167)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    if V <= 65:
                        EFi_PM10 = 0.3861 * (V**-0.5093)
                        EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                    else:
                        EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                        EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_NH3 = 0.001
                elif car_birth == 2005:
                    EFi_CO = 0.7392 * (V**-0.7524)
                    EFi_HC = 0.0989 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0839 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0839 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2006 <= car_birth <= 2010:
                    EFi_CO = 0.5775 * (V**-0.7524)
                    EFi_HC = 0.0825 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0420 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0420 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2011 <= car_birth <= 2015:
                    EFi_CO = 0.5141 * (V**-0.6792)
                    EFi_HC = 0.3713 * (V**-0.7513)
                    EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)        #     EFi_PM10 = 0.0009 * (V**0.0416) 
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416)   #  EFi_PM2_5 = k * 0.0009 * (V**0.0416)
                    EFi_NH3 = 0.0019
                elif 2016 <= car_birth:
                    EFi_CO = 0.4574 * (V**-0.5215)
                    EFi_HC = 0.1300 * (V**-0.7265)
                    EFi_NOx = 2.7702 * (V**-0.3869)
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)           #     EFi_PM10 = 0.0009 * (V**0.0416)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416)      #     EFi_PM2_5 = k * 0.0009 * (V**0.0416)
                    EFi_NH3 = 0.0019
            elif (car_type == '승용') and (car_size == '중형'):
                if car_birth <= 2004:
                    EFi_CO = 5.9672 * (V**-0.9534)
                    EFi_HC = 0.6523 * (V**-1.0167)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    if V <= 65:
                        EFi_PM10 = 0.3861 * (V**-0.5093)
                        EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                    else:
                        EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                        EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_NH3 = 0.001
                elif car_birth == 2005:
                    EFi_CO = 0.6930 * (V**-0.7524)
                    EFi_HC = 0.1865 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0723 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0723 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2006 <= car_birth <= 2010:
                    EFi_CO = 0.5414 * (V**-0.7524)
                    EFi_HC = 0.0927 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0396 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0396 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2011 <= car_birth <= 2015:
                    EFi_CO = 0.5141 * (V**-0.6792)
                    EFi_HC = 0.3713 * (V**-0.7513)
                    EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)            #     EFi_PM10 = 0.0009 * (V**0.0416)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416)      #     EFi_PM2_5 = k * 0.0009 * (V**0.0416)
                    EFi_NH3 = 0.0019
                elif 2016 <= car_birth:
                    EFi_CO = 0.4574 * (V**-0.5215)
                    EFi_HC = 0.1300 * (V**-0.7265)
                    EFi_NOx = 2.7702 * (V**-0.3869)
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)              #     EFi_PM10 = 0.0009 * (V**0.0416)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416)         #     EFi_PM2_5 = k * 0.0009 * (V**0.0416)
                    EFi_NH3 = 0.0019
            elif (car_type == '승용') and (car_size == '대형'):
                if car_birth <= 2004:
                    EFi_CO = 5.9672 * (V**-0.9534)
                    EFi_HC = 0.6523 * (V**-1.0167)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    if V <= 65:
                        EFi_PM10 = 0.3861 * (V**-0.5093)
                        EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                    else:
                        EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                        EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_NH3 = 0.001
                elif car_birth == 2005:
                    EFi_CO = 0.9609 * (V**-0.7524)
                    EFi_HC = 0.1865 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0723 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0723 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2006 <= car_birth <= 2010:
                    EFi_CO = 0.7507 * (V**-0.7524)
                    EFi_HC = 0.1554 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0361 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0361 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2011 <= car_birth <= 2015:
                    EFi_CO = 0.5141 * (V**-0.6792)
                    EFi_HC = 0.3713 * (V**-0.7513)
                    EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)              #     EFi_PM10 = 0.0009 * (V**0.0416)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416)         #     EFi_PM2_5 = k * 0.0009 * (V**0.0416)
                    EFi_NH3 = 0.0019
                elif 2016 <= car_birth:
                    EFi_CO = 0.4574 * (V**-0.5215)
                    EFi_HC = 0.1300 * (V**-0.7265)
                    EFi_NOx = 2.7702 * (V**-0.3869)
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)              # EFi_PM10 = 0.0009 * (V**0.0416)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416)         # EFi_PM2_5 = k * 0.0009 * (V**0.0416)
                    EFi_NH3 = 0.0019
            elif (car_type == '승합') and (car_size == '소형'):
                if car_birth <= 1990:
                    if V <= 65.4:
                        EFi_CO = 3.4539 * (V**-0.4266)
                    else:
                        EFi_CO = 0.0051 * V + 0.2212
                    EFi_HC = 0.9835 * (V**-0.5096)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 1.1412 * (V**-0.4324)
                    EFi_PM2_5 = k * 1.1412 * (V**-0.4324)
                elif 1991 <= car_birth <= 1995:
                    if V <= 65.4:
                        EFi_CO = 3.4539 * (V**-0.4266)
                    else:
                        EFi_CO = 0.0051 * V + 0.2212
                    EFi_HC = 1.6313 * (V**-0.7298)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.5999 * (V**-0.3294)
                    EFi_PM2_5 = k * 0.5999 * (V**-0.3294)
                elif 1996 <= car_birth <= 1997:
                    if V <= 65.4:
                        EFi_CO = 3.4539 * (V**-0.4266)
                    else:
                        EFi_CO = 0.0051 * V + 0.2212
                    EFi_HC = 1.1293 * (V**-0.6588)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.6408 * (V**-0.3596)
                    EFi_PM2_5 = k * 0.6408 * (V**-0.3596)
                elif 1998 <= car_birth <= 1999:
                    EFi_CO = 3.7564 * (V**-0.5175)
                    EFi_HC = 1.1293 * (V**-0.6588)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.5168 * (V**-0.3596)
                    EFi_PM2_5 = k * 0.5168 * (V**-0.3596)
                elif 2000 <= car_birth <= 2003:
                    EFi_CO = 3.7564 * (V**-0.5175)
                    EFi_HC = 1.1293 * (V**-0.6588)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.2894 * (V**-0.3596)
                    EFi_PM2_5 = k * 0.2894 * (V**-0.3596)
                elif 2004 <= car_birth <= 2007:
                    EFi_CO = 3.2797 * (V**-0.8887)
                    EFi_HC = 0.1807 * (V**-0.6588)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.2067 * (V**-0.3596)
                    EFi_PM2_5 = k * 0.2067 * (V**-0.3596)
                elif 2008 <= car_birth <= 2011:
                    if V <= 65.4:
                        EFi_CO = 4.222 * (V**-1.4035)
                    else:
                        EFi_CO = 0.011666 * (V**0.09222)
                    if V <= 97.3:
                        EFi_HC = 0.829 * (V**-1.0961)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.3111 * (V**-0.5125)
                    EFi_PM2_5 = k * 0.3111 * (V**-0.5125)
                elif 2012 <= car_birth:
                    if V <= 65.4:
                        EFi_CO = 4.222 * (V**-1.4035)
                    else:
                        EFi_CO = 0.011666 * (V**0.09222)
                    if V <= 97.3:
                        EFi_HC = 0.829 * (V**-1.0961)
                    if V <= 65.4:
                        EFi_NOx = 2.0217 * (V**-0.2645)
                    else:
                        EFi_NOx = 0.0271 * (V**0.7596)
                    EFi_PM10 = 0.1119 * (V**-0.5125)
                    EFi_PM2_5 = k * 0.1119 * (V**-0.5125)
                EFi_NH3 = 0.001
            elif (car_type == '승합') and (car_size == '중형'):
                if car_birth <= 1995:
                    EFi_CO = 32.550 * (V**-0.4944)
                    EFi_HC = 15.753 * (V**-0.5912)
                    if V < 80:
                        EFi_NOx = 40.692 * (V**-0.5590)
                    else:
                        EFi_NOx = -0.0023 * (V**2) + 0.5381 * V - 23.590
                    EFi_PM10 = 5.4886 * (V**-0.5911)
                    EFi_PM2_5 = k * 5.4886 * (V**-0.5911)
                elif 1996 <= car_birth <= 1997:
                    EFi_CO = 16.410 * (V**-0.3790)
                    EFi_HC = 4.2324 * (V**-0.3926)
                    if V < 80:
                        EFi_NOx = 22.804 * (V**-0.4660)
                    else:
                        EFi_NOx = -0.0021 * (V**2) + 0.4430 * V - 18.730
                    EFi_PM10 = 1.6593 * (V**-0.3935)
                    EFi_PM2_5 = k * 1.6593 * (V**-0.3935)
                elif 1998 <= car_birth <= 2000:
                    EFi_CO = 16.410 * (V**-0.3790)
                    EFi_HC = 4.2324 * (V**-0.3926)
                    if V < 80:
                        EFi_NOx = 25.708 * (V**-0.4772)
                    else:
                        EFi_NOx = 0.0019 * (V**2) - 0.2628 * V + 12.145
                    EFi_PM10 = 1.6593 * (V**-0.3935)
                    EFi_PM2_5 = k * 1.6593 * (V**-0.3935)
                elif 2001 <= car_birth <= 2004:
                    EFi_CO = 16.378 * (V**-0.5340)
                    EFi_HC = 5.8477 * (V**-0.5466)
                    if V < 80:
                        EFi_NOx = 25.436 * (V**-0.4656)
                    else:
                        EFi_NOx = 0.0008 * (V**2) - 0.0482 * V + 1.8424
                    EFi_PM10 = 1.2848 * (V**-0.4715)
                    EFi_PM2_5 = k * 1.2848 * (V**-0.4715)
                elif 2005 <= car_birth <= 2007:
                    EFi_CO = 15.256 * (V**-0.7448)
                    EFi_HC = 2.0502 * (V**-0.6504)
                    EFi_NOx = 15.001 * (V**-0.4528)
                    EFi_PM10 = 0.2979 * (V**-0.4008)
                    EFi_PM2_5 = k * 1.0457 * (V**-0.4527)
                elif 2008 <= car_birth <= 2010:
                    EFi_CO = 8.1771 * (V**-0.7725)
                    EFi_HC = 1.2991 * (V**-0.6538)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                        EFi_PM10 = 0.0539 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                        EFi_PM10 = 2.0000*(10**-4) * V - 7.5600*(10**-3)
                        EFi_PM2_5 = k * 2.0000*(10**-4) * V - 7.5600*(10**-3)
                elif 2011 <= car_birth <= 2014:
                    EFi_CO = 4.5201 * (V**-0.7279)
                    EFi_HC = 1.6826 * (V**-0.8045)
                    if V <= 64.7:
                        EFi_NOx = 17.2485 * (V**-0.4040)
                        EFi_PM10 = 0.0469 * (V**-0.4674)
                        EFi_PM2_5 = k * 0.0469 * (V**-0.4674)
                    else:
                        EFi_NOx = 1.1797 * (V**0.2308)
                        EFi_PM10 = 1.6800*(10**-4) * V - 5.1600*(10**-3)
                        EFi_PM2_5 = k * 1.6800*(10**-4) * V - 5.1600*(10**-3)
                elif 2015 <= car_birth:
                    EFi_CO = 7.4065 * (V**-0.5995)
                    EFi_HC = 2.4562 * (V**-1.3145)
                    EFi_NOx = 42.7393 * (V**-1.2949)
                    if V <= 64.7:
                        EFi_PM10 = 0.0081 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0081 * (V**-0.5182)
                    else:
                        EFi_PM10 = 3.0000*(10**-5) * V - 0.0011
                        EFi_PM2_5 = k * 3.0000*(10**-5) * V - 0.0011
                EFi_NH3 = 0.001
            elif (car_type == '승합') and (car_size == '대형'):
                if car_birth <= 1995:
                    EFi_CO = 28.205 * (V**-0.5337)
                    EFi_HC = 6.1146 * (V**-0.4979)
                    EFi_NOx = 41.346 * (V**-0.3645)
                    EFi_PM10 = 5.2158 * (V**-0.5048)
                    EFi_PM2_5 = k * 5.2158 * (V**-0.5048)
                    EFi_NH3 = 0.003
                elif 1996 <= car_birth <= 1997:
                    EFi_CO = 23.205 * (V**-0.5425)
                    EFi_HC = 6.5657 * (V**-0.5431)
                    EFi_NOx = 42.1379 * (V**-0.3786)
                    EFi_PM10 = 2.4911 * (V**-0.4149)
                    EFi_PM2_5 = k * 2.4911 * (V**-0.4149)
                    EFi_NH3 = 0.003
                elif 1998 <= car_birth <= 2000:
                    EFi_CO = 23.205 * (V**-0.5425)
                    EFi_HC = 6.5657 * (V**-0.5431)
                    EFi_NOx = 42.1379 * (V**-0.3786)
                    EFi_PM10 = 1.4432 * (V**-0.3870)
                    EFi_PM2_5 = k * 1.4432 * (V**-0.3870)
                    EFi_NH3 = 0.003
                elif car_birth == 2001:
                    EFi_CO = 21.348 * (V**-0.5806)
                    EFi_HC = 6.6390 * (V**-0.5760)
                    EFi_NOx = 36.7191 * (V**-0.3548)
                    EFi_PM10 = 0.9375 * (V**-0.3910)
                    EFi_PM2_5 = k * 0.9375 * (V**-0.3910)
                    EFi_NH3 = 0.003
                elif 2002 <= car_birth <= 2004:
                    EFi_CO = 21.348 * (V**-0.5806)
                    EFi_HC = 6.6390 * (V**-0.5760)
                    EFi_NOx = 36.7191 * (V**-0.3548)
                    EFi_PM10 = 1.1507 * (V**-0.4804)
                    EFi_PM2_5 = k * 1.1507 * (V**-0.4804)
                    EFi_NH3 = 0.003
                elif 2005 <= car_birth <= 2007:
                    EFi_CO = 9.6452 * (V**-0.5291)
                    EFi_HC = 3.2339 * (V**-0.7436)
                    EFi_NOx = 30.5870 * (V**-0.3548)
                    if V <= 80:
                        EFi_PM10 = 0.4657 * (V**-0.5634)
                        EFi_PM2_5 = k * 0.4657 * (V**-0.5634)
                    else:
                        EFi_PM10 = 0.0014 * (V**0.7970)
                        EFi_PM2_5 = k * 0.0014 * (V**0.7970)
                    EFi_NH3 = 0.003
                elif 2008 <= car_birth <= 2010:
                    EFi_CO = 6.8493 * (V**-0.6506)
                    EFi_HC = 1.7177 * (V**-0.6781)
                    EFi_NOx = 40.7564 * (V**-0.4757)
                    EFi_PM10 = 0.2418 * (V**-0.4727)
                    EFi_PM2_5 = k * 0.2418 * (V**-0.4727)
                    EFi_NH3 = 0.003
                elif 2011 <= car_birth <= 2014:
                    EFi_CO = 5.4607 * (V**-0.2990)
                    EFi_HC = 0.8863 * (V**-0.6933)
                    EFi_NOx = 40.3729 * (V**-0.5386)
                    EFi_PM10 = 0.2125 * (V**-0.4650)
                    EFi_PM2_5 = k * 0.2125 * (V**-0.4650)
                    EFi_NH3 = 0.003
                elif 2015 <= car_birth:
                    EFi_CO = 11.4415 * (V**-0.8036)
                    EFi_HC = 0.6774 * (V**-0.8321)
                    EFi_NOx = 112.1229 * (V**-1.6393)
                    EFi_PM10 = 0.0363 * (V**-0.4727)
                    EFi_PM2_5 = k * 0.0363 * (V**-0.4727)
                    EFi_NH3 = 0.007
            elif (car_type == '화물') and (car_size == '소형'):
                if car_birth <= 1990:
                    EFi_CO = 4.5854 * (V**-0.3613)
                    EFi_HC = 0.4840 * (V**-0.2756)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.8117 * (V**-0.4071)
                    EFi_PM2_5 = k * 0.8117 * (V**-0.4071)
                elif 1991 <= car_birth <= 1995:
                    EFi_CO = 3.4774 * (V**-0.3483)
                    EFi_HC = 0.4844 * (V**-0.3288)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.6188 * (V**-0.4540)
                    EFi_PM2_5 = k * 0.6188 * (V**-0.4540)
                elif 1996 <= car_birth <= 1997:
                    EFi_CO = 3.3934 * (V**-0.3837)
                    EFi_HC = 0.4955 * (V**-0.3393)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.7037 * (V**-0.5357)
                    EFi_PM2_5 = k * 0.7037 * (V**-0.5357)
                elif 1998 <= car_birth <= 1999:
                    EFi_CO = 4.0896 * (V**-0.6083)
                    EFi_HC = 0.6122 * (V**-0.5684)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.6157 * (V**-0.5357)
                    EFi_PM2_5 = k * 0.6157 * (V**-0.5357)
                elif 2000 <= car_birth <= 2003:
                    EFi_CO = 4.0896 * (V**-0.6083)
                    EFi_HC = 0.6122 * (V**-0.5684)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.4838 * (V**-0.5357)
                    EFi_PM2_5 = k * 0.4838 * (V**-0.5357)
                elif 2004 <= car_birth <= 2007:
                    EFi_CO = 3.2797 * (V**-0.8887)
                    EFi_HC = 0.1807 * (V**-0.6588)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.2067 * (V**-0.3596)
                    EFi_PM2_5 = k * 0.2067 * (V**-0.3596)
                elif 2008 <= car_birth <= 2011:
                    if V <= 65.4:
                        EFi_CO = 4.222 * (V**-1.4035)
                    else:
                        EFi_CO = 0.01166 * (V**0.09222)
                    if V <= 97.3:
                        EFi_HC = 0.829 * (V**-1.0961)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.3111 * (V**-0.5125)
                    EFi_PM2_5 = k * 0.3111 * (V**-0.5125)
                elif 2012 <= car_birth <= 2016:
                    if V <= 65.4:
                        EFi_CO = 4.222 * (V**-1.4035)
                        EFi_NOx = 2.0217 * (V**-0.2645)
                    else:
                        EFi_CO = 0.01166 * (V**0.09222)
                        EFi_NOx = 0.0271 * (V**0.7596)
                    if V <= 97.3:
                        EFi_HC = 0.829 * (V**-1.0961)
                    EFi_PM10 = 0.1119 * (V**-0.5125)
                    EFi_PM2_5 = k * 0.1119 * (V**-0.5125)
                EFi_NH3 = 0.001
            elif (car_type == '화물') and (car_size == '중형'):
                if car_birth <= 1995:
                    EFi_CO = 16.769 * (V**-0.3772)
                    EFi_HC = 6.7755 * (V**-0.5003)
                    EFi_NOx = 24.915 * (V**-0.3942)
                    EFi_PM10 = 3.6772 * (V**-0.5514)
                    EFi_PM2_5 = k * 3.6772 * (V**-0.5514)
                elif 1996 <= car_birth <= 2000:
                    EFi_CO = 21.057 * (V**-0.4958)
                    EFi_HC = 6.7532 * (V**-0.5711)
                    EFi_NOx = 25.022 * (V**-0.4240)
                    EFi_PM10 = 3.5285 * (V**-0.5962)
                    EFi_PM2_5 = k * 3.5285 * (V**-0.5962)
                elif car_birth == 2001:
                    EFi_CO = 23.501 * (V**-0.6100)        # EFi_CO = 23.501 * (V**-0.6100)       
                    EFi_HC = 6.8738 * (V**-0.5913)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 1.4444 * (V**-0.4824)
                    EFi_PM2_5 = k * 1.4444 * (V**-0.4824)
                elif 2002 <= car_birth <= 2004:
                    EFi_CO = 23.501 * (V**-0.6100)        # EFi_CO = 23.501 * (V**-0.6100)   
                    EFi_HC = 6.8738 * (V**-0.5913)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 1.0432 * (V**-0.4992)
                    EFi_PM2_5 = k * 1.0432 * (V**-0.4992)
                elif 2005 <= car_birth <= 2007:
                    EFi_CO = 15.256 * (V**-0.7448)
                    EFi_HC = 2.0502 * (V**-0.6504)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 0.2979 * (V**-0.4008)
                    EFi_PM2_5 = k * 0.2979 * (V**-0.4008)
                elif 2008 <= car_birth <= 2010:
                    EFi_CO = 8.1771 * (V**-0.7725)
                    EFi_HC = 1.2991 * (V**-0.6538)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                        EFi_PM10 = 0.0539 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                        EFi_PM10 = 0.0002 * V - 0.00756
                        EFi_PM2_5 = k * 0.0002 * V - 0.00756
                elif 2011 <= car_birth <= 2014:
                    EFi_CO = 4.5201 * (V**-0.7279)
                    EFi_HC = 1.6826 * (V**-0.8045)
                    if V <= 64.7:
                        EFi_NOx = 17.2485 * (V**-0.4040)
                        EFi_PM10 = 0.0469 * (V**-0.4674)
                        EFi_PM2_5 = k * 0.0469 * (V**-0.4674)
                    else:
                        EFi_NOx = 1.1797 * (V**0.2308)
                        EFi_PM10 = 0.000168 * V - 0.00516
                        EFi_PM2_5 = k * 0.000168 * V - 0.00516
                elif 2015 <= car_birth:
                    EFi_CO = 7.4065 * (V**-0.5995)
                    EFi_HC = 2.4562 * (V**-1.3145)
                    EFi_NOx = 42.7393 * (V**-1.2949)
                    if V <= 64.7:
                        EFi_PM10 = 0.0081 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0081 * (V**-0.5182)
                    else:
                        EFi_PM10 = 3.0000 * (10**-5) * V - 0.0011
                        EFi_PM2_5 = k * 3.0000 * (10**-5) * V - 0.0011
                EFi_NH3 = 0.001
            elif car_type == '특수':
                if car_birth <= 1995:
                    EFi_CO = 16.769 * (V**-0.3772)
                    EFi_HC = 6.7755 * (V**-0.5003)
                    EFi_NOx = 24.915 * (V**-0.3942)
                    EFi_PM10 = 3.6772 * (V**-0.5514)
                    EFi_PM2_5 = k * 3.6772 * (V**-0.5514)
                    EFi_NH3 = 0.003
                elif 1996 <= car_birth <= 2000:
                    EFi_CO = 21.057 * (V**-0.4958)
                    EFi_HC = 6.7532 * (V**-0.5711)
                    EFi_NOx = 25.022 * (V**-0.4240)
                    EFi_PM10 = 3.5285 * (V**-0.5962)
                    EFi_PM2_5 = k * 3.5285 * (V**-0.5962)
                    EFi_NH3 = 0.003
                elif car_birth == 2001:
                    EFi_CO = 25.501 * (V**-0.6100)
                    EFi_HC = 6.8738 * (V**-0.5913)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 1.4444 * (V**-0.4824)
                    EFi_PM2_5 = k * 1.4444 * (V**-0.4824)
                    EFi_NH3 = 0.003
                elif 2002 <= car_birth <= 2004:
                    EFi_CO = 23.501 * (V**-0.6100)             # EFi_CO = 23.501 * (V**-0.6100)
                    EFi_HC = 6.8738 * (V**-0.5913)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 1.0432 * (V**-0.4992)
                    EFi_PM2_5 = k * 1.0432 * (V**-0.4992)
                    EFi_NH3 = 0.003
                elif 2005 <= car_birth <= 2007:
                    EFi_CO = 15.256 * (V**-0.7448)
                    EFi_HC = 2.0502 * (V**-0.6504)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 0.2979 * (V**-0.4008)
                    EFi_PM2_5 = k * 0.2979 * (V**-0.4008)
                    EFi_NH3 = 0.003
                elif 2008 <= car_birth <= 2009:
                    EFi_CO = 8.1771 * (V**-0.7725)
                    EFi_HC = 1.2991 * (V**-0.6538)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                        EFi_PM10 = 0.0539 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                        EFi_PM10 = 0.0002 * V - 0.00756
                        EFi_PM2_5 = k * 0.0002 * V - 0.00756
                    EFi_NH3 = 0.003
                elif car_birth == 2010:
                    EFi_CO = 8.1771 * (V**-0.7725)
                    EFi_HC = 1.2991 * (V**-0.6538)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                        EFi_PM10 = 0.0539 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                        EFi_PM10 = 0.0002 * V - 0.00756
                        EFi_PM2_5 = k * 0.0002 * V - 0.00756
                    EFi_NH3 = 0.011
                elif 2011 <= car_birth <= 2014:
                    EFi_CO = 4.5201 * (V**-0.7279)
                    EFi_HC = 1.6826 * (V**-0.8045)
                    if V <= 64.7:
                        EFi_NOx = 17.2485 * (V**-0.4040)
                        EFi_PM10 = 0.0469 * (V**-0.4674)
                        EFi_PM2_5 = k * 0.0469 * (V**-0.4674)
                    else:
                        EFi_NOx = 1.1797 * (V**0.2308)
                        EFi_PM10 = 0.000168 * V - 0.00516
                        EFi_PM2_5 = k * 0.000168 * V - 0.00516
                    EFi_NH3 = 0.011
                elif 2015 <= car_birth:
                    EFi_CO = 7.4065 * (V**-0.5995)
                    EFi_HC = 2.4562 * (V**-1.3145)
                    EFi_NOx = 42.7393 * (V**-1.2949)
                    if V <= 64.7:
                        EFi_PM10 = 0.0081 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0081 * (V**-0.5182)
                    else:
                        EFi_PM10 = 3.0000 * (10**-5) * V - 0.0011
                        EFi_PM2_5 = k * 3.0000 * (10**-5) * V - 0.0011
                    EFi_NH3 = 0.007
            elif (car_type == '화물') and (car_size == '대형'):
                if car_birth <= 1995:
                    EFi_CO = 30.402 * (V**-0.4685)
                    EFi_HC = 15.75 * (V**-0.582)
                    EFi_NOx = 117.49 * (V**-0.365)
                    EFi_PM10 = 7.6212 * (V**-0.4183)
                    EFi_PM2_5 = k * 7.6212 * (V**-0.4183)
                    EFi_NH3 = 0.003
                elif 1996 <= car_birth <= 1997:
                    EFi_CO = 18.101 * (V**-0.3454)
                    EFi_HC = 10.301 * (V**-0.5856)
                    EFi_NOx = 94.319 * (V**-0.4061)
                    EFi_PM10 = 6.0264 * (V**-0.4627)
                    EFi_PM2_5 = k * 6.0264 * (V**-0.4627)
                    EFi_NH3 = 0.003
                elif 1998 <= car_birth <= 2000:
                    EFi_CO = 18.101 * (V**-0.3454)
                    EFi_HC = 10.301 * (V**-0.5856)
                    EFi_NOx = 94.319 * (V**-0.4061)
                    EFi_PM10 = 4.873 * (V**-0.4382)
                    EFi_PM2_5 = k * 4.873 * (V**-0.4382)
                    EFi_NH3 = 0.003
                elif 2001 <= car_birth <= 2004:
                    EFi_CO = 28.399 * (V**-0.5999)
                    EFi_HC = 10.031 * (V**-0.5828)
                    EFi_NOx = 85.301 * (V**-0.4023)
                    EFi_PM10 = 3.7541 * (V**-0.4055)
                    EFi_PM2_5 = k * 3.7541 * (V**-0.4055)
                    EFi_NH3 = 0.003
                elif 2005 <= car_birth <= 2007:
                    EFi_CO = 52.136 * (V**-0.8618)
                    EFi_HC = 3.7878 * (V**-0.5425)
                    EFi_NOx = 107.5 * (V**-0.5679)
                    EFi_PM10 = 2.6847 * (V**-0.6112)
                    EFi_PM2_5 = k * 2.6847 * (V**-0.6112)
                    EFi_NH3 = 0.003
                elif 2008 <= car_birth <= 2009:
                    EFi_CO = 6.8493 * (V**-0.6506)
                    EFi_HC = 1.7177 * (V**-0.6781)
                    EFi_NOx = 40.7564 * (V**-0.4757)
                    EFi_PM10 = 0.2418 * (V**-0.4727)
                    EFi_PM2_5 = k * 0.2418 * (V**-0.4727)
                    EFi_NH3 = 0.003
                elif 2009 <= car_birth <= 2010:
                    EFi_CO = 6.8493 * (V**-0.6506)
                    EFi_HC = 1.7177 * (V**-0.6781)
                    EFi_NOx = 40.7564 * (V**-0.4757)
                    EFi_PM10 = 0.2418 * (V**-0.4727)
                    EFi_PM2_5 = k * 0.2418 * (V**-0.4727)
                    EFi_NH3 = 0.011
                elif 2011 <= car_birth <= 2014:
                    EFi_CO = 5.4607 * (V**-0.2990)
                    EFi_HC = 0.8863 * (V**-0.6933)
                    EFi_NOx = 40.3729 * (V**-0.5386)
                    EFi_PM10 = 0.2125 * (V**-0.4650)
                    EFi_PM2_5 = k * 0.2125 * (V**-0.4650)
                    EFi_NH3 = 0.011
                elif 2015 <= car_birth:
                    EFi_CO = 4.3762 * (V**-0.4550)
                    EFi_HC = 0.3627 * (V**-0.7071)
                    EFi_NOx = 18.0405 * (V**-1.0986)
                    EFi_PM10 = 0.0363 * (V**-0.4727)
                    EFi_PM2_5 = k * 0.0363 * (V**-0.4727)
                    EFi_NH3 = 0.007
        elif fuel == 'LPG':
            if (car_type == '승용') and (car_size == '경형'):
                if car_birth <= 1996:
                    if V <= 45:
                        EFi_CO = 22.498 * (V**-0.6579)
                    else:
                        EFi_CO = 0.0006 * (V**2) + 0.0004 * V + 0.8272
                    EFi_HC = 12.961 * (V**-0.8364)
                    EFi_NOx = 4.0131 * (V**-0.3236)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1997 <= car_birth <= 1999:
                    if V <= 45:
                        EFi_CO = 19.887 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                    EFi_HC = 2.2714 * (V**-0.7830)
                    EFi_NOx = 1.8528 * (V**-0.3889)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2000 <= car_birth <= 2002:
                    if V <= 45:
                        EFi_CO = 19.887 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                    EFi_HC = 2.2714 * (V**-0.7830)
                    EFi_NOx = 5.8289 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2003 <= car_birth <= 2005:
                    if V <= 45:
                        EFi_CO = 19.887 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                    EFi_HC = 1.1073 * (V**-0.7830)
                    EFi_NOx = 5.8289 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2006 <= car_birth <= 2007:
                    if V <= 45:
                        EFi_CO = 8.9904 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0002 * (V**2) + 0.0457 * V - 1.1403
                    EFi_HC = 0.3549 * (V**-0.7830)
                    EFi_NOx = 0.7228 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif car_birth == 2008:
                    if V <= 79.6:
                        EFi_CO = 0.7693 * (V**-0.7666)
                        EFi_HC = 0.1063 * (V**-1.0745)
                    else:
                        EFi_CO = 5.0000 * (10**-16) * (V**7.2766)
                        EFi_HC = 1.0000 * (10**-15) * (V**6.2696)
                    EFi_NOx = -4.0000 * (10**-6) * (V**2) + 6.0000 * (10**-4) * V + 5.5000 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2009 <= car_birth <= 2011:
                    if V <= 79.6:
                        EFi_CO = 0.7059 * (V**-0.7666)
                        EFi_HC = 0.0974 * (V**-1.0745)
                    else:
                        EFi_CO = 4.5878 * (10**-16) * (V**7.2766)
                        EFi_HC = 9.1667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.7333 * (10**-6) * (V**2) + 5.6000 * (10**-4) * V + 5.1333 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2012 <= car_birth <= 2013:
                    if V <= 79.6:
                        EFi_CO = 0.6830 * (V**-0.7666)
                        EFi_HC = 0.0943 * (V**-1.0745)
                    else:
                        EFi_CO = 4.4393 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.8667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.6373 * (10**-6) * (V**2) + 5.4560 * (10**-4) * V + 5.0013 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif car_birth == 2014:
                    if V <= 79.6:
                        EFi_CO = 0.6602 * (V**-0.7666)
                        EFi_HC = 0.0911 * (V**-1.0745)
                    else:
                        EFi_CO = 4.2909 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.5667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.5413 * (10**-6) * (V**2) + 5.3120 * (10**-4) * V + 4.8693 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2015 <= car_birth:
                    if V <= 79.6:
                        EFi_CO = 0.6374 * (V**-0.7666)
                        EFi_HC = 0.0879 * (V**-1.0745)
                    else:
                        EFi_CO = 4.1425 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.2667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.4453 * (10**-6) * (V**2) + 5.1680 * (10**-4) * V + 4.7373 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
            elif (car_type == '승용') and (car_size == '소형'):
                if car_birth <= 1990:
                    EFi_CO = 72.338 * (V**-0.7587)
                    EFi_HC = 26.532 * (V**-1.0423)
                    EFi_NOx = 9.3011 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1991 <= car_birth == 1996:
                    EFi_CO = 72.338 * (V**-0.7587)
                    EFi_HC = 101.79 * (V**-1.6823)
                    EFi_NOx = 11.287 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1997 <= car_birth <= 1999:
                    EFi_CO = 44.956 * (V**-1.0085)
                    EFi_HC = 11.173 * (V**-1.3927)
                    EFi_NOx = 7.5371 * (V**-0.7864)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2000 <= car_birth <= 2002:
                    EFi_CO = 44.956 * (V**-1.0085)
                    EFi_HC = 11.173 * (V**-1.3927)
                    EFi_NOx = 4.7108 * (V**-0.7864)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2003 <= car_birth <= 2005:
                    EFi_CO = 44.956 * (V**-1.0085)
                    EFi_HC = 3.2821 * (V**-1.3927)
                    EFi_NOx = 4.7108 * (V**-0.7864)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2006 <= car_birth <= 2007:
                    EFi_CO = 39.362 * (V**-1.0085)
                    EFi_HC = 2.8981 * (V**-1.3927)
                    EFi_NOx = 1.8419 * (V**-0.7864)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif car_birth == 2008:
                    if V <= 79.6:
                        EFi_CO = 0.7693 * (V**-0.7666)
                        EFi_HC = 0.1063 * (V**-1.0745)
                    else:
                        EFi_CO = 5.0000 * (10**-16) * (V**7.2766)
                        EFi_HC = 1.0000 * (10**-15) * (V**6.2696)
                    EFi_NOx = -4.0000 * (10**-6) * (V**2) + 6.0000 * (10**-4) * V + 5.5000 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2009 <= car_birth <= 2011:
                    if V <= 79.6:
                        EFi_CO = 0.7059 * (V**-0.7666)
                        EFi_HC = 0.0974 * (V**-1.0745)
                    else:
                        EFi_CO = 4.5878 * (10**-16) * (V**7.2766)
                        EFi_HC = 9.1667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.7333 * (10**-6) * (V**2) + 5.6000 * (10**-4) * V + 5.1333 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2012 <= car_birth <= 2013:
                    if V <= 79.6:
                        EFi_CO = 0.6830 * (V**-0.7666)
                        EFi_HC = 0.0943 * (V**-1.0745)
                    else:
                        EFi_CO = 4.4393 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.8667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.6373 * (10**-6) * (V**2) + 5.4560 * (10**-4) * V + 5.0013 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif car_birth == 2014:
                    if V <= 79.6:
                        EFi_CO = 0.6602 * (V**-0.7666)
                        EFi_HC = 0.0911 * (V**-1.0745)
                    else:
                        EFi_CO = 4.2909 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.5667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.5413 * (10**-6) * (V**2) + 5.3120 * (10**-4) * V + 4.8693 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2015 <= car_birth:
                    if V <= 79.6:
                        EFi_CO = 0.6374 * (V**-0.7666)
                        EFi_HC = 0.0879 * (V**-1.0745)
                    else:
                        EFi_CO = 4.1425 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.2667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.4453 * (10**-6) * (V**2) + 5.1680 * (10**-4) * V + 4.7373 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                if V <= 60:
                    EFi_NH3 = 0.0095
                elif 60 < V <= 90:
                    EFi_NH3 = 0.0082
                elif V > 90:
                    EFi_NH3 = 0.022
            elif ( (car_type == '승용') and ( (car_size == '중형') or (car_size == '대형') ) or ( (car_type == '특수') and (car_size == '중형') ) ): # 최 확인(2023.04.20) : 특수 중형 -> 승용 중형 조건으로 계산
                if car_birth <= 1990:
                    EFi_CO = 72.338 * (V**-0.7587)
                    EFi_HC = 26.532 * (V**-1.0423)
                    EFi_NOx = 9.3011 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1991 <= car_birth == 1996:
                    EFi_CO = 29.825 * (V**-0.6771)
                    EFi_HC = 26.520 * (V**-1.4041)
                    EFi_NOx = 11.287 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1997 <= car_birth <= 1999:
                    EFi_CO = 29.825 * (V**-0.6771)
                    EFi_HC = 4.7595 * (V**-0.9512)
                    EFi_NOx = 12.562 * (V**-0.8606)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2000 <= car_birth <= 2005:
                    EFi_CO = 17.829 * (V**-0.6778)
                    EFi_HC = 6.3668 * (V**-1.2849)
                    EFi_NOx = 8.8952 * (V**-0.8119)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2006 <= car_birth <= 2007:
                    EFi_CO = 73.022 * (V**-1.2078)
                    EFi_HC = 4.4166 * (V**-1.5356)
                    EFi_NOx = 2.0280 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif car_birth == 2008:
                    if V <= 79.6:
                        EFi_CO = 0.7693 * (V**-0.7666)
                        EFi_HC = 0.1063 * (V**-1.0745)
                    else:
                        EFi_CO = 5.0000 * (10**-16) * (V**7.2766)
                        EFi_HC = 1.0000 * (10**-15) * (V**6.2696)
                    EFi_NOx = -4.0000 * (10**-6) * (V**2) + 6.0000 * (10**-4) * V + 5.5000 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2009 <= car_birth <= 2011:
                    if V <= 79.6:
                        EFi_CO = 0.7059 * (V**-0.7666)
                        EFi_HC = 0.0974 * (V**-1.0745)
                    else:
                        EFi_CO = 4.5878 * (10**-16) * (V**7.2766)
                        EFi_HC = 9.1667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.7333 * (10**-6) * (V**2) + 5.6000 * (10**-4) * V + 5.1333 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2012 <= car_birth <= 2013:
                    if V <= 79.6:
                        EFi_CO = 0.6830 * (V**-0.7666)
                        EFi_HC = 0.0943 * (V**-1.0745)
                    else:
                        EFi_CO = 4.4393 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.8667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.6373 * (10**-6) * (V**2) + 5.4560 * (10**-4) * V + 5.0013 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif car_birth == 2014:
                    if V <= 79.6:
                        EFi_CO = 0.6602 * (V**-0.7666)
                        EFi_HC = 0.0911 * (V**-1.0745)
                    else:
                        EFi_CO = 4.2909 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.5667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.5413 * (10**-6) * (V**2) + 5.3120 * (10**-4) * V + 4.8693 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2015 <= car_birth:
                    if V <= 79.6:
                        EFi_CO = 0.6374 * (V**-0.7666)
                        EFi_HC = 0.0879 * (V**-1.0745)
                    else:
                        EFi_CO = 4.1425 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.2667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.4453 * (10**-6) * (V**2) + 5.1680 * (10**-4) * V + 4.7373 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                if V <= 60:
                    EFi_NH3 = 0.0095
                elif 60 < V <= 90:
                    EFi_NH3 = 0.0082
                elif V > 90:
                    EFi_NH3 = 0.022
            elif ( (car_type == '승합') and (car_size == '경형') ) or ( (car_type == '화물') and (car_size == '경형') ):
                if car_birth <= 1996:
                    if V <= 45:
                        EFi_CO = 22.498 * (V**-0.6579)
                    else:
                        EFi_CO = 0.0006 * (V**2) + 0.0004 * V + 0.8272
                    EFi_HC = 12.961 * (V**-0.8364)
                    EFi_NOx = 4.0131 * (V**-0.3236)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1997 <= car_birth == 1999:
                    if V <= 45:
                        EFi_CO = 19.887 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                    EFi_HC = 2.2714 * (V**-0.7830)
                    EFi_NOx = 1.8528 * (V**-0.3889)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2000 <= car_birth <= 2002:
                    if V <= 45:
                        EFi_CO = 19.887 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                    EFi_HC = 2.2714 * (V**-0.7830)
                    EFi_NOx = 5.8289 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2003 <= car_birth <= 2005:
                    if V <= 45:
                        EFi_CO = 19.887 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                    EFi_HC = 1.1073 * (V**-0.7830)
                    EFi_NOx = 5.8289 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2006 <= car_birth:
                    if V <= 45:
                        EFi_CO = 8.9904 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0002 * (V**2) + 0.0457 * V - 1.1403
                    EFi_HC = 0.3549 * (V**-0.7830)
                    EFi_NOx = 0.7228 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                if V <= 60:
                    EFi_NH3 = 0.0095
                elif 60 < V <= 90:
                    EFi_NH3 = 0.0082
                elif V > 90:
                    EFi_NH3 = 0.022
            elif (car_type == '승합') and (car_size == '소형'):
                if car_birth <= 1990:
                    EFi_CO = 72.338 * (V**-0.7587)
                    EFi_HC = 26.532 * (V**-1.0423)
                    EFi_NOx = 9.3011 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1991 <= car_birth == 1996:
                    EFi_CO = 29.825 * (V**-0.6771)
                    EFi_HC = 26.520 * (V**-1.4041)
                    EFi_NOx = 11.287 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1997 <= car_birth <= 1999:
                    EFi_CO = 29.825 * (V**-0.6771)
                    EFi_HC = 4.7595 * (V**-0.9512)
                    EFi_NOx = 12.562 * (V**-0.8606)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2000 <= car_birth <= 2005:
                    EFi_CO = 17.829 * (V**-0.6778)
                    EFi_HC = 6.3668 * (V**-1.2849)
                    EFi_NOx = 8.8952 * (V**-0.8119)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2006 <= car_birth:
                    if V <= 65.4:
                        EFi_CO = 6.1701 * (V**-1.0719)
                        EFi_HC = 2.3782 * (V**-1.9979)
                    else:
                        EFi_CO = 3.0000 * (10**-10) * (V**4.5809)
                        EFi_HC = 5.0000 * (10**-10) * (V**3.4895)
                    if V <= 97.3:
                        EFi_NOx = 0.4758 * (V**-1.0665)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                if V <= 60:
                    EFi_NH3 = 0.0095
                elif 60 < V <= 90:
                    EFi_NH3 = 0.0082
                elif V > 90:
                    EFi_NH3 = 0.022
            # 최 확인(2023.04.20) : 화물 중형 -> 승합 중형 조건, 화물 대형 -> 승합 대형 조건으로 계산
            elif ( (car_type == '승합') and ( (car_size == '중형') or (car_size == '대형') ) ) or ( (car_type == '화물') and ((car_size == '소형') or (car_size == '중형') or (car_size == '대형')) ):
                if car_birth <= 1990:
                    EFi_CO = 72.338 * (V**-0.7587)
                    EFi_HC = 26.532 * (V**-1.0423)
                    EFi_NOx = 9.3011 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1991 <= car_birth == 1996:
                    EFi_CO = 29.825 * (V**-0.6771)
                    EFi_HC = 26.520 * (V**-1.4041)
                    EFi_NOx = 11.287 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1997 <= car_birth <= 1999:
                    EFi_CO = 29.825 * (V**-0.6771)
                    EFi_HC = 4.7595 * (V**-0.9512)
                    EFi_NOx = 12.562 * (V**-0.8606)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2000 <= car_birth <= 2005:
                    EFi_CO = 17.829 * (V**-0.6778)
                    EFi_HC = 6.3668 * (V**-1.2849)
                    EFi_NOx = 8.8952 * (V**-0.8119)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2006 <= car_birth:
                    if V <= 65.4:
                        EFi_CO = 16.491 * (V**-1.4713)
                        EFi_HC = 1.9218 * (V**-1.8418)
                    else:
                        EFi_CO = 3.0000 * (10**-13) * (V**6.0619)
                        EFi_HC = 3.0000 * (10**-9) * (V**3.0639)
                    if V <= 97.3:
                        EFi_NOx = 0.1246 * (V**-0.441)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                if V <= 60:
                    EFi_NH3 = 0.0095
                elif 60 < V <= 90:
                    EFi_NH3 = 0.0082
                elif V > 90:
                    EFi_NH3 = 0.022
        # else:
        #     # 값 고민 : 1
        #     # EFi_CO, EFi_HC, EFi_NOx, EFi_PM10, EFi_PM2_5 = np.nan, np.nan, np.nan, np.nan, np.nan
        #     EFi_CO, EFi_HC, EFi_NOx, EFi_PM10, EFi_PM2_5 = 1, 1, 1, 1, 1

        if V < 60:
            EFi_COm += EFi_CO*0.34
            EFi_HCm += EFi_HC*0.34
            EFi_NOxm += EFi_NOx*0.34
            EFi_PM10m += EFi_PM10*0.34
            EFi_PM2_5m += EFi_PM2_5*0.34
            EFi_NH3m += EFi_NH3*0.34
        elif 60 <= V < 90:
            EFi_COm += EFi_CO*0.33
            EFi_HCm += EFi_HC*0.33
            EFi_NOxm += EFi_NOx*0.33
            EFi_PM10m += EFi_PM10*0.33
            EFi_PM2_5m += EFi_PM2_5*0.33
            EFi_NH3m += EFi_NH3*0.33
        elif V > 90:
            EFi_COm += EFi_CO*0.33
            EFi_HCm += EFi_HC*0.33
            EFi_NOxm += EFi_NOx*0.33
            EFi_PM10m += EFi_PM10*0.33
            EFi_PM2_5m += EFi_PM2_5*0.33
            EFi_NH3m += EFi_NH3*0.33

    # 배출계수(EFi) 리스트에 저장
    EFi_CO_list.append(EFi_COm)
    EFi_HC_list.append(EFi_HCm)
    EFi_NOx_list.append(EFi_NOxm)
    EFi_PM10_list.append(EFi_PM10m)
    EFi_PM2_5_list.append(EFi_PM2_5m)
    EFi_NH3_list.append(EFi_NH3m)

100%|██████████| 25117098/25117098 [05:39<00:00, 73944.99it/s]


In [32]:
df2.shape[0]

25117098

In [33]:
len(EFi_CO_list), len(EFi_HC_list), len(EFi_NOx_list), len(EFi_PM10_list), len(EFi_PM2_5_list), len(EFi_NH3_list)

(25117098, 25117098, 25117098, 25117098, 25117098, 25117098)

In [34]:
# 28.1s
df2['EFi_CO'] = EFi_CO_list
df2['EFi_HC'] = EFi_HC_list
df2['EFi_NOx'] = EFi_NOx_list
df2['EFi_PM10'] = EFi_PM10_list
df2['EFi_PM2_5'] = EFi_PM2_5_list
df2['EFi_NH3'] = EFi_NH3_list

In [35]:
check_EFi_col = ['EFi_CO', 'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5']

### 배출량 계산

In [36]:
DF_col + check_DF_col + check_R_col

['fuel',
 '차종',
 '차종유형',
 '차량연식',
 '차령',
 'DF_CO',
 'DF_HC',
 'DF_PM',
 'DF_NOx',
 'R_CO',
 'R_HC',
 'R_PM']

In [37]:
df2[DF_col + check_DF_col + check_R_col + check_EFi_col].head()

,fuel,차종,차종유형,차량연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx,R_CO,R_HC,R_PM,EFi_CO,EFi_HC,EFi_NOx,EFi_PM10,EFi_PM2_5
0,휘발유,승용,중형,2023,0,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.06,0.00,0.01,0.00,0.00
1,휘발유,승용,중형,2023,0,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.06,0.00,0.01,0.00,0.00
2,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.08,0.00,0.01,0.00,0.00
3,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.08,0.00,0.01,0.00,0.00
4,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.08,0.00,0.01,0.00,0.00


In [38]:
# E = VKT * (EFi / 1000) * DF * (1 - R / 100)
df2['E_HOT_CO'] = df2['VKT'] * (df2['EFi_CO'] / 1000) * df2['DF_CO'] * (1 - df2['R_CO'] / 100)
df2['E_HOT_HC'] = df2['VKT'] * (df2['EFi_HC'] / 1000) * df2['DF_HC'] * (1 - df2['R_HC'] / 100)
df2['E_HOT_NOx'] = df2['VKT'] * (df2['EFi_NOx'] / 1000) * df2['DF_NOx'] * (1 - 0 / 100)
df2['E_HOT_PM10'] = df2['VKT'] * (df2['EFi_PM10'] / 1000) * df2['DF_PM'] * (1 - df2['R_PM'] / 100)
df2['E_HOT_PM2_5'] = df2['VKT'] * (df2['EFi_PM2_5'] / 1000) * df2['DF_PM'] * (1 - df2['R_PM'] / 100)

In [39]:
check_E_HOT_col = ['E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10', 'E_HOT_PM2_5']

In [40]:
df2[DF_col + check_DF_col + check_R_col + check_EFi_col + check_E_HOT_col].head()

,fuel,차종,차종유형,차량연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx,R_CO,...,EFi_CO,EFi_HC,EFi_NOx,EFi_PM10,EFi_PM2_5,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5
0,휘발유,승용,중형,2023,0,1.00,1.00,1.00,1.00,0.00,...,0.06,0.00,0.01,0.00,0.00,0.78,0.02,0.16,0.01,0.01
1,휘발유,승용,중형,2023,0,1.00,1.00,1.00,1.00,0.00,...,0.06,0.00,0.01,0.00,0.00,0.80,0.02,0.16,0.01,0.01
2,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00,0.00,...,0.08,0.00,0.01,0.00,0.00,1.32,0.02,0.21,0.01,0.01
3,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00,0.00,...,0.08,0.00,0.01,0.00,0.00,1.32,0.02,0.21,0.01,0.01
4,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00,0.00,...,0.08,0.00,0.01,0.00,0.00,1.29,0.02,0.21,0.01,0.01


In [41]:
df2.shape

(25117098, 38)

## 자동차-엔진 미가열(Cold-start) 배출

### 배출량 식
$$E_{COLD :i, j} = \beta_{i, j} \times N_j \times M_j \times e^{HOT}_j \times (e^{COLD} / e^{HOT} \vert_{i, j} - 1)$$

- $E_{COLD :i, j}$ : 차종 $j$에서 배출되는 오염물질 $i$의 엔진미가열 배출량
- $\beta_{i, j}$ : 차종 $j$의 엔진미가열 상태의 주행거리 분율
- $N_j$ : 차종 $j$의 수
- $M_j$ : 차종 $j$의 주행거리
- $e^{HOT}_j$ : 차종 $j$의 엔진가열 상태에서의 배출계수
- $e^{COLD} / e^{HOT} \vert_{i, j}$ : 차종 $j$의 엔진가열상태 대비 엔진미가열 상태에서의 배출 비율

### 베타($\beta$)
- 1회 평균주행거리(1 trip length)와 대기온도, 자동차 이용 패턴을 고려
    - 1회 평균주행거리 정보는 모든 차종에 대하여 각각 적용하는 방안은 수집자료의 한계로 국내 연구결과를 바탕으로 수도권 지역 승용차의 1회 평균 주행거리 12.35km를 적용
    - 이때 과대산정 가능성을 염두하여, 도시지역의 택시를 제외한 승용차(경형, 소형, 중형, 대형), 승합차(경형, 소형), RV(소형, 중형)에 적용

    - 1회 주행거리 : 12.35 km (2002년 기준)
    - 미가열 배출 적용 차종 : 승용차(경형, 소형, 중형, 대형) / 승합차(경형, 소형) / RV(소형, 중형)
    - 미가열 배출 적용 지역 : 도시지역(고속도로 구간 제외)

- 산정식

-|Factor $\beta$의 산정식
-|-
$Estimated$ $l_{trip}$|$0.647-0.025 \times l_{trip} - (0.00974 - 0.000385 \times l_{trip}) \times T_a$
$Measured$ $l_{trip}$|$0.698-0.051 \times l_{trip} - (0.01051 - 0.000770 \times l_{trip}) \times T_a$

- $T_a$ : 대기온도
- $Estimated$ $l_{trip}$ : 유럽평균(1985) 약 11.8km / 유럽평균(1998) 약 12.4km(Andre et al., 1998)

In [42]:
l_trip = 12.35 # 국립환경과학원(2007), 도로 이동오염원 대기오염 배출량 산정방법 개선 및 장래 배출량 예측방법 연구
Ta = 12.4 # e-나라지표 (https://www.index.go.kr/unity/potal/main/EachDtlPageDetail.do?idx_cd=1400) 2002년 년 평균 기온

In [43]:
df2['Beta(Estimated)'] = 0.647 - 0.025 * l_trip - (0.00974 - 0.000385 * l_trip) * Ta
# df2['Beta(Measured)'] = 0.698 - 0.051 * l_trip - (0.01051 - 0.000770 * l_trip) * Ta

In [44]:
df2['Beta(Estimated)'].head()

0   0.28
1   0.28
2   0.28
3   0.28
4   0.28
Name: Beta(Estimated), dtype: float64

In [45]:
df2['Beta(Estimated)'].isnull().sum()

0

### $e^{COLD} / e^{HOT}$

In [46]:
df2['fuel'].unique()

array(['휘발유', 'LPG', '경유'], dtype=object)

In [47]:
df2['차량연식'].describe()

count   25117098.00
mean        2014.36
std            6.19
min         1900.00
25%         2011.00
50%         2015.00
75%         2019.00
max         2023.00
Name: 차량연식, dtype: float64

In [48]:
# 1m 31.7s
# 휘발유(자동 제어)
df2.loc[(df2['fuel'] == '휘발유') & (df2['차량연식'] < 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_CO'] = 3.7 - 0.09 * Ta
df2.loc[(df2['fuel'] == '휘발유') & (df2['차량연식'] >= 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_CO'] = 9.04 - 0.09 * Ta
df2.loc[(df2['fuel'] == '휘발유') & (df2['차량연식'] < 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_NOx'] = 1.14 - 0.006 * Ta
df2.loc[(df2['fuel'] == '휘발유') & (df2['차량연식'] >= 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_NOx'] = 3.66 - 0.006 * Ta
df2.loc[(df2['fuel'] == '휘발유') & (df2['차량연식'] < 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_HC'] = 2.8 - 0.06 * Ta
df2.loc[(df2['fuel'] == '휘발유') & (df2['차량연식'] >= 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_HC'] = 12.59 - 0.06 * Ta
df2.loc[df2['fuel'] == '휘발유', 'eCOLD/eHOT_PM'] = 1
# 경유
df2.loc[(df2['fuel'] == '경유') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_CO'] = 1.9 - 0.03 * Ta
df2.loc[(df2['fuel'] == '경유') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_NOx'] = 1.3 - 0.013 * Ta
df2.loc[(df2['fuel'] == '경유') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_HC'] = 3.1 - 0.09 * Ta
df2.loc[(df2['fuel'] == '경유') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_PM'] = 3.1 - 0.1 * Ta
# LPG
df2.loc[(df2['fuel'] == 'LPG') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_CO'] = 3.66 - 0.09 * Ta
df2.loc[(df2['fuel'] == 'LPG') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_NOx'] = 0.98 - 0.006 * Ta
df2.loc[(df2['fuel'] == 'LPG') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_HC'] = 2.24 - 0.06 * Ta
df2.loc[(df2['fuel'] == 'LPG') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_PM'] = 1

In [49]:
check_ecoldhot_col = ['eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx', 'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM']
df2[check_ecoldhot_col].isnull().sum()

eCOLD/eHOT_CO     4282101
eCOLD/eHOT_NOx    4282101
eCOLD/eHOT_HC     4282101
eCOLD/eHOT_PM     4248868
dtype: int64

In [50]:
df2[check_ecoldhot_col] = df2[check_ecoldhot_col].fillna(1)
df2[check_ecoldhot_col].isnull().sum()

eCOLD/eHOT_CO     0
eCOLD/eHOT_NOx    0
eCOLD/eHOT_HC     0
eCOLD/eHOT_PM     0
dtype: int64

In [51]:
df2.loc[df2['eCOLD/eHOT_CO'] < 0, 'eCOLD/eHOT_CO'].shape

(0,)

In [52]:
df2.loc[df2['eCOLD/eHOT_NOx'] < 0, 'eCOLD/eHOT_NOx'].shape

(0,)

In [53]:
df2.loc[df2['eCOLD/eHOT_HC'] < 0, 'eCOLD/eHOT_HC'].shape

(0,)

In [54]:
df2.loc[df2['eCOLD/eHOT_PM'] < 0, 'eCOLD/eHOT_PM'].shape

(0,)

### 배출량 계산

In [55]:
df2.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군', '일일평균주행거리',
       '저감장치부착유무', 'VKT', 'DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx', 'R_CO', 'R_HC',
       'R_PM', 'EFi_CO', 'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5',
       'EFi_NH3', 'E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10',
       'E_HOT_PM2_5', 'Beta(Estimated)', 'eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx',
       'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM'],
      dtype='object')

#### 🔺 계산식 고민
$$E_{COLD :i, j} = \beta_{i, j} \times N_j \times M_j \times e^{HOT}_j \times (e^{COLD} / e^{HOT} \vert_{i, j} - 1)$$
- $M_j$ : 차종 $j$의 주행거리
    - 1년 주행거리로 계산? 
        - VKT
    - 1회 주행거리로 계산?
        - l_trip

In [56]:
#         E      =             B          *     M      *    e(HOT)     * ( e(COLD)/e(HOT)      - 1 )
df2['E_COLD_CO'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_CO'] * (df2['eCOLD/eHOT_CO'] - 1)
df2['E_COLD_NOx'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_NOx'] * (df2['eCOLD/eHOT_NOx'] - 1)
df2['E_COLD_HC'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_HC'] * (df2['eCOLD/eHOT_HC'] - 1)
df2['E_COLD_PM10'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_PM10'] * (df2['eCOLD/eHOT_PM'] - 1)
df2['E_COLD_PM2_5'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_PM2_5'] * (df2['eCOLD/eHOT_PM'] - 1)

In [57]:
df2.loc[df2['E_COLD_NOx'] < 0, ['fuel', 'Beta(Estimated)', 'VKT', 'EFi_NOx', 'eCOLD/eHOT_NOx', 'E_COLD_NOx']].shape

(1758697, 6)

In [58]:
df2.loc[df2['E_COLD_NOx'] < 0, ['fuel', 'Beta(Estimated)', 'VKT', 'EFi_NOx', 'eCOLD/eHOT_NOx', 'E_COLD_NOx']].head()

,fuel,Beta(Estimated),VKT,EFi_NOx,eCOLD/eHOT_NOx,E_COLD_NOx
5,LPG,0.28,12811.50,0.02,0.91,-7.03
6,LPG,0.28,13578.00,0.02,0.91,-7.45
7,LPG,0.28,12775.00,0.02,0.91,-7.01
8,LPG,0.28,13906.50,0.02,0.91,-7.63
9,LPG,0.28,14855.50,0.02,0.91,-8.15


In [59]:
check_E_cold_col = ['E_COLD_CO', 'E_COLD_NOx', 'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5']
df2[check_E_HOT_col + check_E_cold_col].head()

,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,E_COLD_CO,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5
0,0.78,0.02,0.16,0.01,0.01,1492.93,113.83,51.06,0.00,0.00
1,0.80,0.02,0.16,0.01,0.01,1526.76,116.41,52.21,0.00,0.00
2,1.32,0.02,0.21,0.01,0.01,2518.00,149.62,67.11,0.00,0.00
3,1.32,0.02,0.21,0.01,0.01,2518.00,149.62,67.11,0.00,0.00
4,1.29,0.02,0.21,0.01,0.01,2474.58,147.04,65.95,0.00,0.00


In [60]:
df2[check_E_HOT_col + check_E_cold_col].tail()

,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,E_COLD_CO,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5
25117093,111.87,36.94,114.11,12.88,11.85,0.00,0.00,0.00,0.00,0.00
25117094,234.20,57.98,260.10,25.71,23.65,0.00,0.00,0.00,0.00,0.00
25117095,125.34,41.45,585.35,38.36,35.30,0.00,0.00,0.00,0.00,0.00
25117096,98.86,24.48,109.80,10.85,9.98,0.00,0.00,0.00,0.00,0.00
25117097,133.88,44.28,625.23,40.98,37.70,0.00,0.00,0.00,0.00,0.00


In [61]:
df2[check_E_cold_col].isnull().sum()

E_COLD_CO       0
E_COLD_NOx      0
E_COLD_HC       0
E_COLD_PM10     0
E_COLD_PM2_5    0
dtype: int64

## 자동차-휘발유 증발 배출

### 배출량 식
- 3가지
    - 주간증발손실(Diurnal loss)
    - 고온증발손실(Hot and warm soak)
    - 주행손실(Running loss)
- 현재('19 배출량 기준) 휘발유 증발 배출원
    - 주간증발손실, 주행손실 고려
    - 우리나라 휘발유 차량 대부분이 방지설비로 카본 캐니스터를 장착하고 있으므로 $e^{S,HOT}$, $e^{S,WARM}$, 고온증발손실(hot and warm soak) 배출은 없는 것으로 가정

---

$$E_{EVA, VOC:j} = 365 \times N_j \times (e^d + S^c + S^{fi}) + R$$
$S^c = (1-q)(p \times x \times e^{S, HOT} + w \times x \times e^{S,WARM})$

$S^{fi} = q \times e^{fi} \times x$
$$R = m_j(p \times e^{R,HOT} + w \times e^{R,WARM})$$

- $E_{EVA, VOC:j}$ : 차종 j의 증발손실에 의한 VOC 배출량(g/yr)
- $N_j$ : 차종 j의 휘발유 사용 차량 등록대수(대)
- $e^d$ : 금속탱크를 가지고 있는 휘발유 차량의 일중 VOC 배출량(g/day-대)
- $S^c$ : Carburetor 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $S^{fi}$ : Fuel injection 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $R$ : Running loss에 의한 휘발유 차량의 연중 배출량(g/yr)
- $q$ : Fuel injection 휘발유 차량의 비율
- $p$ : 가열 엔진 상태로 끝나는 trip의 비율
- $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
- $w$ : 미가열 엔진 상태료 끝나는 trip의 비율
- $e^{S,HOT}$ : Hot soak emission의 평균 배출계수
- $e^{S,WARM}$ : Cold and warm soak emission의 평균 배출계수
- $e^{fi}$ : Fuel injection 휘발유 차량의 평균 hot and warm soak 배출계수
- $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수
- $e^{R,WARM}$ : 휘발유 차량의 평균 warm running loss 배출계수
- $m_j$ : 차종 j의 연간 총 주행거리
- $V_j$ : 차종 j의 1대당 평균 연간 주행거리

### 배출계수

#### 🔺 수정 중

#### $e^d$ Diurnal(g/day)
- $e^d$ : 금속탱크를 가지고 있는 휘발유 차량의 일중 VOC 배출량(g/day-대)

##### 🔺 RVP(증기압)
- 참고 COPERT 3(p.80) Table 6.10: Gasoline fuel specifications
    - summer : 60
    - winter : 70
    - 평균 : 65
- 수정 사항(2023.04.20, 최)
    - 국내 과학기술원, 환경부 자료 찾아서 사용 추천

In [62]:
RVP = 65

##### ❗ t_a(기온)
- t_a, t_min, t_rise
    - 참고 COPERT 3(p.72) Table 5.32: Summary of emission factors for estimating evaporative emissions of gasoline vehicles(all RVp in kPa, all temperatures in °C)
        - t_a = (t_max + t_min) / 2
        - t_rise = t_max - t_min
    - 기상자료개방포털(2002년) 연평균 기온
        - https://data.kma.go.kr/stcs/grnd/grndTaList.do?pgmNo=70
        - 7.5°C(최저), 12.4°C(평균), 17.9°C(최고)
    - 기상자료개방포털(2022년) 연평균 기온
        - https://data.kma.go.kr/stcs/grnd/grndTaList.do?pgmNo=70
        - 8°C(최저), 12.9°C(평균), 18.6°C(최고)

In [63]:
t_min = 8 # 평균 최저 온도
t_max = 18.6
t_a = (t_min + t_max) / 2
t_rise = 10.6 # 평균 상승 온도

In [64]:
df2.loc[df2['차량연식'] < 1991, 'e_d'] = 9.1 * np.exp( 0.0158 * (RVP - 61.2) + 0.0574 * (t_min - 22.5) + 0.0614 * (t_rise - 11.7) )
df2.loc[df2['차량연식'] >= 1991, 'e_d'] = 0.2 * (9.1 * np.exp( 0.0158 * (RVP - 61.2) + 0.0574 * (t_min - 22.5) + 0.0614 * (t_rise - 11.7) ))

In [65]:
df2['e_d'].isnull().sum()

0

#### $e^{R,HOT}$ Hot running loss(g/km)
- $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수

In [66]:
df2.loc[df2['차량연식'] < 1991, 'e_RHOT'] = 0.136 * np.exp( - 5.967 + 0.04259 * RVP + 0.1773 * t_a)
df2.loc[df2['차량연식'] >= 1991, 'e_RHOT'] = 0.1 * (0.136 * np.exp( - 5.967 + 0.04259 * RVP + 0.1773 * t_a))

In [67]:
df2['e_RHOT'].isnull().sum()

0

#### $e^{S,HOT}$ Hot soak(g/procedure)
- $e^{S,HOT}$ : Hot soak emission의 평균 배출계수

In [68]:
df2.loc[df2['차량연식'] < 1991, 'e_SHOT'] = 3.0042 * np.exp(0.02 * RVP)
df2.loc[df2['차량연식'] >= 1991, 'e_SHOT'] = 0.3 * np.exp(-2.14 + 0.02302 * RVP + 0.09408 * t_a)

#### $e^{S,WARM}$ Waram soak(g/procedure)
- $e^{S,WARM}$ : Cold and warm soak emission의 평균 배출계수

In [69]:
df2.loc[df2['차량연식'] < 1991, 'e_SWARM'] = np.exp(-1.644 + 0.01993 * RVP + 0.07521 * t_a)
df2.loc[df2['차량연식'] >= 1991, 'e_SWARM'] = 0.2 * np.exp(-2.41 + 0.02302 * RVP + 0.09408 * t_a)

#### $e^{fi}$ Warm and hot soak for Fuel injected vehicle(g/procedure)
- $e^{fi}$ : Fuel injection 휘발유 차량의 평균 hot and warm soak 배출계수

In [70]:
df2.loc[df2['차량연식'] < 1991, 'e_fi'] = 0.7
df2.loc[df2['차량연식'] >= 1991, 'e_fi'] = 0

#### $e^{R,WARM}$ Warm running loss(g/km)
- $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수

In [71]:
df2.loc[df2['차량연식'] < 1991, 'e_RWARM'] = 0.1 * np.exp(-5.967 + 0.04259 * RVP + 0.1773 * t_a)
df2.loc[df2['차량연식'] >= 1991, 'e_RWARM'] = 0.1 * (0.1 * np.exp(-5.967 + 0.04259 * RVP + 0.1773 * t_a))

In [72]:
df2['e_RWARM'].isnull().sum()

0

### $R$
$R = m_j(p \times e^{R,HOT} + w \times e^{R,WARM})$
- Running loss에 의한 휘발유 차량의 연중 배출량(g/yr)
- $m_j$ : 차종 j의 연간 총 주행거리
- $p$ : 가열 엔진 상태로 끝나는 trip의 비율
- $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수
- $w$ : 미가열 엔진 상태로 끝나는 trip의 비율
- $e^{R,WARM}$ : 휘발유 차량의 평균 warm running loss 배출계수

#### ❓ 확인 중
- 수정 사항(2023.04.20, 최)
    - 찾는 중

In [73]:
p = 0.5 # 가열 엔진 상태로 끝나는 trip의 비율
w = 0.5 # 미가열 엔진 상태로 끝나는 trip의 비율

In [74]:
df2['R'] = df2['VKT'] * (p * df2['e_RHOT'] + w * df2['e_RWARM'])

In [75]:
df2['R'].isnull().sum()

0

### $S^{fi}$
$S^{fi} = q \times e^{fi} \times x$
- Fuel injection 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $q$ : Fuel injection 휘발유 차량의 비율
- $e^{fi}$ : Fuel injection 휘발유 차량의 평균 hot and warm soak 배출계수
- $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
- $V_j$ : 차종 j의 1대당 평균 연간 주행거리

In [76]:
# 8.9s
not_injection = df2[(df2['fuel'] == '휘발유') & (df2['차량연식'] < 1991)].shape[0]
injection = df2[(df2['fuel'] == '휘발유') & (df2['차량연식'] >= 1991)].shape[0]
# Fuel injection 휘발유 차량의 비율
q = injection / (injection + not_injection)
q

0.9931614094087922

In [77]:
# x = df2['VKT'] / (365 * l_trip)
df2['S_fi'] = q * df2['e_fi'] * ( df2['VKT'] / (365 * l_trip) )

### $S^c$
$S^c = (1-q)(p \times x \times e^{S, HOT} + w \times x \times e^{S,WARM})$
- Carburetor 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $q$ : Fuel injection 휘발유 차량의 비율
- $p$ : 가열 엔진 상태로 끝나는 trip의 비율
- $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
- $e^{S,HOT}$ : Hot soak emission의 평균 배출계수
- $w$ : 미가열 엔진 상태료 끝나는 trip의 비율
- $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
- $V_j$ : 차종 j의 1대당 평균 연간 주행거리
- $e^{S,WARM}$ : Cold and warm soak emission의 평균 배출계수

In [78]:
# x = df2['VKT'] / (365 * l_trip)
df2['S_c'] = (1-q) * (p * (df2['VKT'] / (365 * l_trip)) * df2['e_SHOT'] + w * (df2['VKT'] / (365 * l_trip)) * df2['e_SWARM'])

### $E_{EVA,VOC}$
$E_{EVA, VOC:j} = 365 \times N_j \times (e^d + S^c + S^{fi}) + R$
- $N_j$ : 차종 j의 휘발유 사용 차량 등록대수(대)
- $e^d$ : 금속탱크를 가지고 있는 휘발유 차량의 일중 VOC 배출량(g/day-대)
- $S^c$ : Carburetor 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $S^{fi}$ : Fuel injection 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $R$ : Running loss에 의한 휘발유 차량의 연중 배출량(g/yr)

In [79]:
df2['E_EVA_VOC'] = 365 * df2['e_d'] + df2['S_c'] + df2['S_fi'] + df2['R']

In [80]:
df2['E_EVA_VOC'].isnull().sum()

0

## 배출량 합계
$E_{total}(kg) = E_{HOT}(kg) + E_{COLD}(g) + E_{EVAP}(g)$

In [81]:
df2[df2['E_COLD_NOx'] < 0].shape

(1758697, 58)

In [82]:
# E_COLD_NOx 음수 -> 0으로 처리(2023.04.24 from 최이사님)
df2.loc[df2['E_COLD_NOx'] < 0, 'E_COLD_NOx'] = 0

In [83]:
df2[df2['E_COLD_NOx'] < 0].shape

(0, 58)

In [84]:
check_E_HOT_col, check_E_cold_col

(['E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10', 'E_HOT_PM2_5'],
 ['E_COLD_CO', 'E_COLD_NOx', 'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5'])

In [85]:
df2.loc[(df2['fuel'] == '휘발유') & (df2['E_COLD_PM10'] != 0)].shape

(0, 58)

In [86]:
df2['fuel'].unique()

array(['휘발유', 'LPG', '경유'], dtype=object)

In [87]:
# 26.5s
fuel = '휘발유'
df2.loc[df2['fuel'] == fuel, 'E_CO_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_CO'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_CO']/1000
df2.loc[df2['fuel'] == fuel, 'E_HC_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_HC'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_HC']/1000 + df2.loc[df2['fuel'] == fuel , 'E_EVA_VOC']/1000
df2.loc[df2['fuel'] == fuel, 'E_NOx_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_NOx'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_NOx']/1000
df2.loc[df2['fuel'] == fuel, 'E_PM10_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_PM10'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_PM10']/1000
df2.loc[df2['fuel'] == fuel, 'E_PM2_5_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_PM2_5'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_PM2_5']/1000

In [88]:
# 22.7s
fuel = '경유'
df2.loc[df2['fuel'] == fuel, 'E_CO_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_CO'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_CO']/1000
df2.loc[df2['fuel'] == fuel, 'E_HC_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_HC'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_HC']/1000
df2.loc[df2['fuel'] == fuel, 'E_NOx_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_NOx'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_NOx']/1000
df2.loc[df2['fuel'] == fuel, 'E_PM10_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_PM10'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_PM10']/1000
df2.loc[df2['fuel'] == fuel, 'E_PM2_5_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_PM2_5'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_PM2_5']/1000

In [89]:
# 20.0s
fuel = 'LPG'
df2.loc[df2['fuel'] == fuel, 'E_CO_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_CO'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_CO']/1000
df2.loc[df2['fuel'] == fuel, 'E_HC_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_HC'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_HC']/1000
df2.loc[df2['fuel'] == fuel, 'E_NOx_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_NOx'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_NOx']/1000
df2.loc[df2['fuel'] == fuel, 'E_PM10_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_PM10'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_PM10']/1000
df2.loc[df2['fuel'] == fuel, 'E_PM2_5_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_PM2_5'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_PM2_5']/1000

In [90]:
check_E_col = ['E_CO_total', 'E_HC_total', 'E_NOx_total', 'E_PM10_total', 'E_PM2_5_total']
df2[check_E_col].head()

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
0,2.27,0.42,0.27,0.01,0.01
1,2.32,0.42,0.28,0.01,0.01
2,3.83,0.46,0.36,0.01,0.01
3,3.83,0.46,0.36,0.01,0.01
4,3.77,0.46,0.35,0.01,0.01


In [91]:
df2.loc[df2['fuel'] == '휘발유', check_E_col + ['E_EVA_VOC']].head()

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_EVA_VOC
0,2.27,0.42,0.27,0.01,0.01,352.45
1,2.32,0.42,0.28,0.01,0.01,353.94
2,3.83,0.46,0.36,0.01,0.01,373.08
3,3.83,0.46,0.36,0.01,0.01,373.08
4,3.77,0.46,0.35,0.01,0.01,371.59


In [92]:
df2.loc[df2['fuel'] == '경유', check_E_col].head()

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
19,0.85,0.12,7.98,0.02,0.02
26,13.38,0.41,4.53,0.10,0.10
27,13.53,0.42,4.58,0.10,0.10
28,13.05,0.40,4.42,0.10,0.09
29,13.12,0.40,4.44,0.10,0.09


In [93]:
df2.loc[df2['fuel'] == 'LPG', check_E_col].head()

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
5,1.33,0.03,0.27,0.00,0.00
6,1.41,0.03,0.29,0.00,0.00
7,1.32,0.03,0.27,0.00,0.00
8,1.44,0.03,0.29,0.00,0.00
9,1.54,0.03,0.31,0.00,0.00


In [94]:
df2.loc[(df2['fuel'] == '경유') & (df2['차종'] == '승용'), DF_col + check_E_col].head()

,fuel,차종,차종유형,차량연식,차령,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
19,경유,승용,대형,2023,0,0.85,0.12,7.98,0.02,0.02
42,경유,승용,중형,2023,0,0.84,0.12,7.85,0.02,0.02
43,경유,승용,중형,2023,0,1.15,0.16,10.81,0.03,0.02
44,경유,승용,중형,2023,0,1.15,0.16,10.81,0.03,0.02
49,경유,승용,대형,2023,0,0.87,0.12,8.16,0.02,0.02


In [95]:
df2.loc[(df2['fuel'] == '경유') & (df2['차종'] == '화물'), DF_col + check_E_col].head()

,fuel,차종,차종유형,차량연식,차령,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
26,경유,화물,대형,2023,0,13.38,0.41,4.53,0.10,0.10
27,경유,화물,대형,2023,0,13.53,0.42,4.58,0.10,0.10
28,경유,화물,대형,2023,0,13.05,0.40,4.42,0.10,0.09
29,경유,화물,대형,2023,0,13.12,0.40,4.44,0.10,0.09
60,경유,화물,소형,2022,1,1.32,0.02,0.10,0.01,0.01


In [96]:
df2.loc[(df2['fuel'] == '휘발유') & (df2['차종'] == '승용'), DF_col + check_E_col].head()

,fuel,차종,차종유형,차량연식,차령,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
0,휘발유,승용,중형,2023,0,2.27,0.42,0.27,0.01,0.01
1,휘발유,승용,중형,2023,0,2.32,0.42,0.28,0.01,0.01
2,휘발유,승용,대형,2023,0,3.83,0.46,0.36,0.01,0.01
3,휘발유,승용,대형,2023,0,3.83,0.46,0.36,0.01,0.01
4,휘발유,승용,대형,2023,0,3.77,0.46,0.35,0.01,0.01


In [97]:
df2.loc[(df2['fuel'] == '휘발유') & (df2['차종'] == '화물'), DF_col + check_E_col].head()

,fuel,차종,차종유형,차량연식,차령,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
2829,휘발유,화물,소형,2023,0,2.16,0.43,0.18,0.01,0.01
2830,휘발유,화물,소형,2023,0,2.16,0.43,0.18,0.01,0.01
2831,휘발유,화물,소형,2023,0,2.16,0.43,0.18,0.01,0.01
2832,휘발유,화물,소형,2023,0,2.16,0.43,0.18,0.01,0.01
3124,휘발유,화물,소형,2022,1,2.61,0.46,0.22,0.01,0.01


### issue
- LPG : E_COLD_NOx 값 음수 -> 0으로 처리

In [98]:
df2.loc[df2['fuel'] == 'LPG', DF_col + check_E_HOT_col + check_E_cold_col + check_E_col].head()

,fuel,차종,차종유형,차량연식,차령,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,E_COLD_CO,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
5,LPG,승용,중형,2023,0,0.93,0.03,0.27,0.00,0.00,396.57,0.00,3.51,0.00,0.00,1.33,0.03,0.27,0.00,0.00
6,LPG,승용,중형,2023,0,0.98,0.03,0.29,0.00,0.00,420.29,0.00,3.72,0.00,0.00,1.41,0.03,0.29,0.00,0.00
7,LPG,승용,중형,2023,0,0.93,0.03,0.27,0.00,0.00,395.44,0.00,3.50,0.00,0.00,1.32,0.03,0.27,0.00,0.00
8,LPG,승용,중형,2023,0,1.01,0.03,0.29,0.00,0.00,430.46,0.00,3.81,0.00,0.00,1.44,0.03,0.29,0.00,0.00
9,LPG,승용,중형,2023,0,1.08,0.03,0.31,0.00,0.00,459.84,0.00,4.07,0.00,0.00,1.54,0.03,0.31,0.00,0.00


In [99]:
df2.loc[df2['fuel'] == 'LPG', DF_col + check_E_cold_col + check_E_col].head()

,fuel,차종,차종유형,차량연식,차령,E_COLD_CO,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
5,LPG,승용,중형,2023,0,396.57,0.00,3.51,0.00,0.00,1.33,0.03,0.27,0.00,0.00
6,LPG,승용,중형,2023,0,420.29,0.00,3.72,0.00,0.00,1.41,0.03,0.29,0.00,0.00
7,LPG,승용,중형,2023,0,395.44,0.00,3.50,0.00,0.00,1.32,0.03,0.27,0.00,0.00
8,LPG,승용,중형,2023,0,430.46,0.00,3.81,0.00,0.00,1.44,0.03,0.29,0.00,0.00
9,LPG,승용,중형,2023,0,459.84,0.00,4.07,0.00,0.00,1.54,0.03,0.31,0.00,0.00


In [100]:
df2.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군', '일일평균주행거리',
       '저감장치부착유무', 'VKT', 'DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx', 'R_CO', 'R_HC',
       'R_PM', 'EFi_CO', 'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5',
       'EFi_NH3', 'E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10',
       'E_HOT_PM2_5', 'Beta(Estimated)', 'eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx',
       'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM', 'E_COLD_CO', 'E_COLD_NOx',
       'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5', 'e_d', 'e_RHOT', 'e_SHOT',
       'e_SWARM', 'e_fi', 'e_RWARM', 'R', 'S_fi', 'S_c', 'E_EVA_VOC',
       'E_CO_total', 'E_HC_total', 'E_NOx_total', 'E_PM10_total',
       'E_PM2_5_total'],
      dtype='object')

In [101]:
df2['E_COLD_CO'].dtype

dtype('float64')

In [102]:
df2.loc[df2['E_CO_total'] < 0].shape

(0, 63)

In [103]:
df2.loc[df2['E_HC_total'] < 0].shape

(0, 63)

In [104]:
df2.loc[df2['E_PM10_total'] < 0].shape

(0, 63)

In [105]:
df2.loc[df2['E_PM2_5_total'] < 0].shape

(0, 63)

In [106]:
df2.loc[df2['E_NOx_total'] < 0, ['E_NOx_total', 'E_COLD_NOx']].shape

(0, 2)

## 황산화물 배출량

In [107]:
V

100

In [108]:
df2['차종'].unique()

array(['승용', '화물', '특수', '승합', nan], dtype=object)

In [109]:
df2['차종유형'].unique()

array(['중형', '대형', '경형', '소형', nan], dtype=object)

In [110]:
# 46.2s
fcc_list = []
for fone, cd, ty in tqdm(df2[['fuel', '차종', '차종유형']].values):
    fccm = 0
    for V in [30, 75, 100]:
        if fone == '휘발유':
            if cd == '승용':
                fcc = 0.01090 * (V**2) - 1.5100*V + 93.672 # E01
            elif cd == '승합':
                if ty =='경형':
                    fcc = 0.01090 * (V**2) - 1.5100*V + 93.672 # E01
                elif ty == '소형' or ty == '중형' or ty == '대형':
                    fcc = 0.01870 * (V**2) - 2.6974*V + 156.77 # E04
            elif cd == '화물':
                if ty == '경형':   
                    fcc = 0.01090 * (V**2) - 1.5100*V + 93.672 # E01
                elif ty == '소형' or ty == '중형' or ty == '대형':
                    fcc = 0.01870 * (V**2) - 2.6974*V + 156.77 # E04
            elif cd == '특수':
                if ty == '소형' or ty == '중형' or ty == '대형':
                    fcc = 0.01870 * (V**2) - 2.6974*V + 156.77 # E04
        elif fone == '경유':
            if cd == '승용':
                fcc = 0.00790 * (V**2) - 1.3123*V + 83.660 # E12
            elif cd == '승합':
                if ty == '경형' or ty == '소형':
                    fcc = 0.00790 * (V**2) - 1.3123*V + 83.660 # E12
                elif ty == '중형':
                    if V <= 60:
                        fcc = 1425.2 * (V**(-0.7593)) # E17
                    else:
                        fcc = 0.0082 * (V**2) - 0.0430*V + 60.12 # E17
                elif ty == '대형':
                    if V <= 60:
                        fcc = 1919.0 * (V**(-0.5396)) # E16
                    else:
                        fcc = 0.0447 * (V**2) - 7.072*V + 478 # E16
            elif cd == '화물':
                if ty == '경형' or ty == '소형':
                    fcc = 0.00790 * (V**2) - 1.3123*V + 83.660 # E12
                elif ty == '중형':
                    if V <= 60:
                        fcc = 1068.4 * (V**(-0.4905)) # E18
                    else:
                        fcc = 0.0126 * (V**2) - 0.6589*V + 141.2 # E18
                elif ty == '대형':
                    if V <= 60:
                        fcc = 1595.1 * (V**(-0.4744)) # E19
                    else:
                        fcc = 0.0382 * (V**2) - 5.1630*V + 399.3 # E19

            elif cd == '특수':
                if ty == '소형':
                    if V <= 60:
                        fcc = 1068.4 * (V**(-0.4905)) # E18
                    else:
                        fcc = 0.0126 * (V**2) - 0.6589*V + 141.2 # E18
                if ty == '중형' or ty == '대형':
                    if V <= 60:
                        fcc = 1595.1 * (V**(-0.4744)) # E19
                    else:
                        fcc = 0.0382 * (V**2) - 5.1630*V + 399.3 # E19
        elif fone == 'LPG':
            if cd == '승용':
                fcc = 45 # E09
            elif cd == '승합':
                fcc = 45 # E09
            elif cd == '화물':
                if ty == '경형' or ty == '소형' or ty == '중형':
                    fcc = 45 # E09
            elif cd == '특수':
                if ty == '소형' or ty == '중형'or ty == '대형':
                    fcc = 45 # E09
        if V < 60:
            fccm += fcc * 0.34
        elif 60 <= V < 90:
            fccm += fcc * 0.33
        elif V > 90:
            fccm += fcc * 0.33
    fcc_list.append(fccm)

100%|██████████| 25117098/25117098 [01:55<00:00, 218169.91it/s]


In [111]:
df2.shape

(25117098, 63)

In [112]:
len(fcc_list)

25117098

In [113]:
# 5.0s
# 단위 : %
sf_list = []
for fone in tqdm(df2['fuel']):
    if fone == '경유':
        sf = 0.017
    elif fone == '휘발유':
        sf = 0.001
    elif fone == 'LPG':
        sf = 0.015
    sf_list.append(sf)

  2%|▏         | 468124/25117098 [00:00<00:15, 1582193.27it/s]

100%|██████████| 25117098/25117098 [00:13<00:00, 1816060.36it/s]


In [114]:
df2.shape, len(sf_list)

((25117098, 63), 25117098)

In [115]:
# 3.5s
df2['fcc'] = fcc_list
df2['sf'] = sf_list

In [116]:
df2['E_SOx_total'] = df2['VKT'] * df2['fcc'] * df2['sf'] * 2 / (1000*100)

## NH3

In [117]:
df2.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군', '일일평균주행거리',
       '저감장치부착유무', 'VKT', 'DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx', 'R_CO', 'R_HC',
       'R_PM', 'EFi_CO', 'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5',
       'EFi_NH3', 'E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10',
       'E_HOT_PM2_5', 'Beta(Estimated)', 'eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx',
       'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM', 'E_COLD_CO', 'E_COLD_NOx',
       'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5', 'e_d', 'e_RHOT', 'e_SHOT',
       'e_SWARM', 'e_fi', 'e_RWARM', 'R', 'S_fi', 'S_c', 'E_EVA_VOC',
       'E_CO_total', 'E_HC_total', 'E_NOx_total', 'E_PM10_total',
       'E_PM2_5_total', 'fcc', 'sf', 'E_SOx_total'],
      dtype='object')

In [118]:
df2['VKT'].describe()

count   25117098.00
mean       14437.04
std         3300.01
min         2847.00
25%        13030.50
50%        13505.00
75%        14271.50
max        98185.00
Name: VKT, dtype: float64

In [119]:
df2['EFi_NH3'].isnull().sum()

0

In [120]:
df2['EFi_NH3'].describe()

count   25117098.00
mean           0.01
std            0.01
min            0.00
25%            0.00
50%            0.01
75%            0.01
max            0.13
Name: EFi_NH3, dtype: float64

In [121]:
df2['E_NH3_total'] = df2['VKT'] * df2['EFi_NH3'] / 1000

In [122]:
df2['E_NH3_total'].describe()

count   25117098.00
mean           0.14
std            0.20
min            0.01
25%            0.03
50%            0.15
75%            0.16
max            5.29
Name: E_NH3_total, dtype: float64

## CO2

In [123]:
df2.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군', '일일평균주행거리',
       '저감장치부착유무', 'VKT', 'DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx', 'R_CO', 'R_HC',
       'R_PM', 'EFi_CO', 'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5',
       'EFi_NH3', 'E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10',
       'E_HOT_PM2_5', 'Beta(Estimated)', 'eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx',
       'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM', 'E_COLD_CO', 'E_COLD_NOx',
       'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5', 'e_d', 'e_RHOT', 'e_SHOT',
       'e_SWARM', 'e_fi', 'e_RWARM', 'R', 'S_fi', 'S_c', 'E_EVA_VOC',
       'E_CO_total', 'E_HC_total', 'E_NOx_total', 'E_PM10_total',
       'E_PM2_5_total', 'fcc', 'sf', 'E_SOx_total', 'E_NH3_total'],
      dtype='object')

In [124]:
df2[['차량연식', '배기량', '총중량']].dtypes

차량연식      int64
배기량     float64
총중량     float64
dtype: object

In [125]:
df2['차종유형'].unique()

array(['중형', '대형', '경형', '소형', nan], dtype=object)

### 계산식

In [126]:
fc_list = []
for ty, yr, fuel, cc, twg in tqdm(df2[['차종유형', '차량연식', 'fuel', '배기량', '총중량']].values):
    fcm = 0
    for V in [30, 75, 100]:
        if fuel == '휘발유' and ty != '경형' and ty !='대형':
            if yr <= 1971:
                if cc < 1400:
                    if 10 <= V < 60:
                        fc = 521 * V**(-0.554)
                    elif 60 <= V < 80:
                        fc = 55
                    elif 80 <= V <130:
                        fc = 0.386*V + 24143
                    else:
                        fc = 0
                elif 1400 <= cc < 2000:
                    if 10 <= V < 60:
                        fc = 681 * V**(-0.583)
                    elif 60 <= V < 80:
                        fc = 67
                    elif 80 <= V <130:
                        fc = 0.471*V + 29286
                    else:
                        fc = 0
                elif cc >= 2000:
                    if 10 <= V < 60:
                        fc = 979 * V**(-0.628)
                    elif 60 <= V < 80:
                        fc = 80
                    elif 80 <= V <130:
                        fc = 0.414*V + 46867
                    else:
                        fc = 0
                else:
                    fc = 0
            elif 1972 <= yr <= 1977:
                if cc < 1400:
                    if 10 <= V < 60:
                        fc = 595 * V**(-0.63)
                    elif 60 <= V < 130:
                        fc = 95 - 1.324*V + 0.0086*V**2
                    else:
                        fc = 0
                elif 1400 <= cc < 2000:
                    if 10 <= V < 60:
                        fc = 864 * V**(-0.69)
                    elif 60 <= V < 130:
                        fc = 59 - 0.407*V + 0.0042*V**2
                    else:
                        fc = 0
                elif cc >= 2000:
                    if 10 <= V < 60:
                        fc = 1236 * V**(-0.764)
                    elif 60 <= V < 130:
                        fc = 65 - 0.407*V + 0.0042*V**2
                    else:
                        fc = 0
                else:
                    fc = 0
            elif 1978 <= yr <= 1985:
                if cc < 1400:
                    if 10 <= V < 50:
                        fc = 544 * V**(-0.63)
                    elif 50 <= V < 130:
                        fc = 85 - 1.108*V + 0.0077*V**2
                    else:
                        fc = 0
                elif 1400 <= cc < 2000:
                    if 10 <= V < 50:
                        fc = 879 * V**(-0.72)
                    elif 50 <= V < 130:
                        fc = 71 - 0.7032*V + 0.0059*V**2
                    else:
                        fc = 0
                elif cc >= 2000:
                    if 10 <= V < 50:
                        fc = 1224 * V**(-0.756)
                    elif 50 <= V < 130:
                        fc = 111 - 1.333*V + 0.0093*V**2
                    else:
                        fc = 0
                else:
                    fc = 0
            elif 1986 <= yr <= 1992:
                if cc < 1400:
                    if 10 <= V < 17.9:
                        fc = 296.7 - 80.21*ln(V)
                    elif 17.9 <= V < 130:
                        fc = 81.1 - 1.014*V + 0.0068*V**2
                    else:
                        fc = 0
                elif 1400 <= cc < 2000:
                    if 10 <= V < 22.3:
                        fc = 606.1 * V**(-0.667)
                    elif 22.3 <= V < 130:
                        fc = 102.5 - 1.364*V + 0.0086*V**2
                    else:
                        fc = 0
                elif cc >= 2000:
                    if 10 <= V < 59.5:
                        fc = 819.9 * V**(-0.663)
                    elif 59.5 <= V < 130:
                        fc = 41.7 - 0.122*V + 0.0016*V**2
                    else:
                        fc = 0
                else:
                    fc = 0
            elif yr > 1993:
                if cc < 1400:
                    if 10 <= V < 130:
                        fc = 85.55 - 1.383*V + 0.0117*V**2
                elif 1400 <= V <2000:
                    if 10 <= V < 130:
                        fc = 109.6 - 1.98*V + 0.0168*V**2
                    else:
                        fc = 0
                else:
                    fc = 0
            else:
                fc = 0
        elif fuel == '휘발유' and ty == '경형':
            if twg < 3500:
                if 10 <= V < 110:
                    fc = 0.0167*V**2 - 2.649*V + 161.51
                else:
                    fc = 0
            else:
                fc = 0
        elif fuel == '휘발유' and ty == '대형':
            if twg > 3500:
                if V <= 60:
                    fc = 225
                elif 60 < V <= 90:
                    fc = 150
                elif V > 90:
                    fc = 165
                else:
                    fc = 0
            else:
                fc = 0
        elif fuel == '경유' and ty != '경형' and ty != '대형':
            if yr <= 1991:
                if twg < 2500:
                    if 10 <= V < 130:
                        fc = 91.106 - 1.308*V + 0.00871*V**2
            else:
                if twg < 2500:
                    if 10 <= V < 130:
                        fc = 118.489 - 2.084*V + 0.014*V**2
                    else:
                        fc = 0
                else:
                    fc = 0
        elif fuel == '경유' and ty == '경형':
            if twg < 3500:
                if 10 <= V < 110:
                    fc = 0.02113*V**2 - 2.65*V + 148.91
                else:
                    fc = 0
            else:
                fc = 0
        elif fuel == '경유' and ty == '대형':
            if twg > 3500:
                if twg < 7500:
                    if 0 <= V < 47:
                        fc = 1425.2*V**(-0.7593)
                    elif 47 <= V < 100:
                        fc = 0.0082*V**2 - 0.0430*V + 60.12
                    else:
                        fc = 0
                elif 7500 <= twg < 16000:
                    if 0 <= V < 59:
                        fc = 1068.4*V**(-0.4905)
                    elif 59 <= V < 100:
                        fc = 0.0126*V**2 - 0.6589*V + 141.18
                    else:
                        fc = 0
                elif 16000 <= twg < 32000:
                    if 0 <= V < 59:
                        fc = 1595.1*V**(-0.4744)
                    elif 59 <= V < 100:
                        fc = 0.0382*V**2 - 5.1630*V + 399.3
                    else:
                        fc = 0
                elif twg > 32000:
                    if 0 <= V < 58:
                        fc = 1855.7*V**(-0.4367)
                    elif 58 <= V < 100:
                        fc = 0.0765*V**2 - 11.414*V + 720.9
                    else:
                        fc = 0
                else:
                    fc = 0
        elif fuel == 'LPG':
            if yr <= 1991:
                if twg < 2500:
                    if 10 <= V < 130:
                        fc = 0.00720*V**2 - 0.9250*V + 74.625
                    else:
                        fc = 0
                else:
                    fc = 0
            else:
                if twg < 2500:
                    if V <= 60:
                        fc = 59
                    elif 60 < V <= 90:
                        fc = 45
                    elif V > 90:
                        fc = 54
                    else:
                        fc = 0
                else:
                    fc = 0
        else:
            fc = 0

        if V < 60:
            fc = fc*0.34
        elif 60 <= V <= 90:
            fc = fc*0.33
        elif V > 90:
            fc = fc*0.33
        fcm += fc
    fc_list.append(fcm)

100%|██████████| 25117098/25117098 [02:09<00:00, 194474.09it/s]


In [127]:
df2.shape, len(fc_list)

((25117098, 67), 25117098)

In [128]:
df2['FC'] = fc_list

In [129]:
r_HC_list = []
for fuel in tqdm(df2['fuel']):
    if fuel == '휘발유':
        r_HC = 1.8
    elif fuel == '경유':
        r_HC = 2.0
    else:
        r_HC = 1.3 # chatGPT 최이사님(2023.06.22.목)
    r_HC_list.append(r_HC)

  0%|          | 0/25117098 [00:00<?, ?it/s]

100%|██████████| 25117098/25117098 [00:13<00:00, 1830181.82it/s]


In [130]:
df2.shape, len(r_HC_list)

((25117098, 68), 25117098)

In [131]:
df2['r_HC'] = r_HC_list

In [132]:
df2.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군', '일일평균주행거리',
       '저감장치부착유무', 'VKT', 'DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx', 'R_CO', 'R_HC',
       'R_PM', 'EFi_CO', 'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5',
       'EFi_NH3', 'E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10',
       'E_HOT_PM2_5', 'Beta(Estimated)', 'eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx',
       'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM', 'E_COLD_CO', 'E_COLD_NOx',
       'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5', 'e_d', 'e_RHOT', 'e_SHOT',
       'e_SWARM', 'e_fi', 'e_RWARM', 'R', 'S_fi', 'S_c', 'E_EVA_VOC',
       'E_CO_total', 'E_HC_total', 'E_NOx_total', 'E_PM10_total',
       'E_PM2_5_total', 'fcc', 'sf', 'E_SOx_total', 'E_NH3_total', 'FC',
       'r_HC'],
      dtype='object')

In [133]:
df2['E_CO2_total'] = (44011 * ( df2['FC'] / (12011 + 1008 * df2['r_HC'])) - df2['E_CO_total'] / 28011 - df2['E_HC_total'] / 1385 - (df2['E_PM10_total'] + df2['E_PM2_5_total']) / 12011) / 1000

In [134]:
df2['E_CO2_total'].describe()

count   25117098.00
mean           0.07
std            0.12
min           -0.00
25%           -0.00
50%           -0.00
75%            0.17
max           49.49
Name: E_CO2_total, dtype: float64

In [135]:
df2[df2['E_CO2_total'] < 0].shape

(15785180, 70)

### 음수 0으로 처리

In [136]:
df2.loc[df2['E_CO2_total'] < 0, 'E_CO2_total'] = 0

# 통계

## 배출량 통계1
- 조건
    - 승용(경형, 소형, 중형)
    - 화물(소형, 중형)

In [137]:
ad = df2.loc[(df2['차종'] == '승용') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형') | (df2['차종유형'] == '중형')) | (df2['차종'] == '화물') & ((df2['차종유형'] == '소형') | (df2['차종유형'] == '중형'))].reset_index(drop=True)
ad.shape

(17575496, 70)

In [138]:
adgl = ad[(ad['fuel'] == '휘발유') | (ad['fuel'] == 'LPG')]
add = ad[ad['fuel'] == '경유']
ad.shape[0] == adgl.shape[0] + add.shape[0]

True

In [139]:
grp1 = adgl.groupby(['등급'], dropna=False).agg({'E_CO_total':'sum', 'E_HC_total':'sum', 'E_NOx_total':'sum', 'E_PM10_total':'sum', 'E_PM2_5_total':'sum', 'E_SOx_total':'sum', 'E_NH3_total':'sum', 'E_CO2_total':'sum', '차대번호':'count'})
grp1

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차대번호
등급,,,,,,,,,
1,3353576.42,447121.24,426403.61,8046.61,7402.88,101637.14,239431.70,98085.05,1464897
2,51199298.96,4228282.19,2532228.13,52806.59,48589.23,240610.39,1392899.40,616949.22,8142605
3,14934817.89,855383.69,2532611.54,8003.05,7420.24,44057.77,402573.41,57510.59,743317
4,20152676.11,2439355.71,4081780.43,13131.28,12154.53,6806.26,327520.43,26310.80,305946
5,12092105.69,1446128.74,789084.26,451.40,415.29,488.23,28614.71,6471.45,29276
X,383980.30,40906.78,30203.12,480.09,441.68,2228.90,13206.68,4379.23,75078


In [140]:
grp1['CO'] = grp1['E_CO_total'] / grp1['차대번호']
grp1['NOx'] = grp1['E_NOx_total'] / grp1['차대번호']
grp1['SOx'] = grp1['E_SOx_total'] / grp1['차대번호']
grp1['PM10'] = grp1['E_PM10_total'] / grp1['차대번호']
grp1['PM2.5'] = grp1['E_PM2_5_total'] / grp1['차대번호']
grp1['HC'] = grp1['E_HC_total'] / grp1['차대번호']
grp1['NH3'] = grp1['E_NH3_total'] / grp1['차대번호']
grp1['CO2'] = grp1['E_CO2_total'] / grp1['차대번호']
grp1

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차대번호,CO,NOx,SOx,PM10,PM2.5,HC,NH3,CO2
등급,,,,,,,,,,,,,,,,,
1,3353576.42,447121.24,426403.61,8046.61,7402.88,101637.14,239431.70,98085.05,1464897,2.29,0.29,0.07,0.01,0.01,0.31,0.16,0.07
2,51199298.96,4228282.19,2532228.13,52806.59,48589.23,240610.39,1392899.40,616949.22,8142605,6.29,0.31,0.03,0.01,0.01,0.52,0.17,0.08
3,14934817.89,855383.69,2532611.54,8003.05,7420.24,44057.77,402573.41,57510.59,743317,20.09,3.41,0.06,0.01,0.01,1.15,0.54,0.08
4,20152676.11,2439355.71,4081780.43,13131.28,12154.53,6806.26,327520.43,26310.80,305946,65.87,13.34,0.02,0.04,0.04,7.97,1.07,0.09
5,12092105.69,1446128.74,789084.26,451.40,415.29,488.23,28614.71,6471.45,29276,413.04,26.95,0.02,0.02,0.01,49.40,0.98,0.22
X,383980.30,40906.78,30203.12,480.09,441.68,2228.90,13206.68,4379.23,75078,5.11,0.40,0.03,0.01,0.01,0.54,0.18,0.06


In [141]:
grp1_1 = adgl.groupby(['등급', '차종'], dropna=False)['차대번호'].count().unstack('차종')
grp1_1

차종,승용,화물
등급,,
1,1464897.00,NaN
2,8077885.00,64720.00
3,742042.00,1275.00
4,300546.00,5400.00
5,27378.00,1898.00
X,74468.00,610.00


In [142]:
grp2 = add.groupby(['등급', '저감장치부착유무'], dropna=False).agg({'E_CO_total':'sum', 'E_HC_total':'sum', 'E_NOx_total':'sum', 'E_PM10_total':'sum', 'E_PM2_5_total':'sum', 'E_SOx_total':'sum', 'E_NH3_total':'sum', 'E_CO2_total':'sum', '차대번호':'count'})
grp2

E_CO_total  E_HC_total  E_NOx_total  E_PM10_total  E_PM2_5_total   
등급 저감장치부착유무                                                                     
3  NaN       8173773.29  1132422.26  54511823.43     333196.28      305981.30  \
4  유               0.58        2.29       994.60          1.47           1.34   
   NaN        824810.53   272087.92  22430903.77     468057.61      430230.49   
5  유           11039.96    40753.91   4523895.79      28589.85       26801.73   
   NaN      14765897.82  3354099.77  31174567.70    1766112.04     1639439.84   
X  유              11.51        6.22      1881.68         20.47          20.39   
   NaN         74118.31     5972.95    122838.98       2691.80        2508.18   

             E_SOx_total  E_NH3_total  E_CO2_total     차대번호  
등급 저감장치부착유무                                                  
3  NaN        1194307.60    118239.56    535449.77  5213000  
4  유               10.90         0.28         0.50       15  
   NaN         207573.09     13566.04     58273.81   835890  
5  유            49669.36      1241.34      3178.40    72050  
   NaN         315377.72     11202.53     30720.50   673947  
X  유               14.94         1.12        13.06       82  
   NaN           4937.90       404.14      1182.42    19393

In [143]:
grp2['CO'] = grp2['E_CO_total'] / grp2['차대번호']
grp2['NOx'] = grp2['E_NOx_total'] / grp2['차대번호']
grp2['SOx'] = grp2['E_SOx_total'] / grp2['차대번호']
grp2['PM10'] = grp2['E_PM10_total'] / grp2['차대번호']
grp2['PM2.5'] = grp2['E_PM2_5_total'] / grp2['차대번호']
grp2['HC'] = grp2['E_HC_total'] / grp2['차대번호']
grp2['NH3'] = grp2['E_NH3_total'] / grp2['차대번호']
grp2['CO2'] = grp2['E_CO2_total'] / grp2['차대번호']
grp2

E_CO_total  E_HC_total  E_NOx_total  E_PM10_total  E_PM2_5_total   
등급 저감장치부착유무                                                                     
3  NaN       8173773.29  1132422.26  54511823.43     333196.28      305981.30  \
4  유               0.58        2.29       994.60          1.47           1.34   
   NaN        824810.53   272087.92  22430903.77     468057.61      430230.49   
5  유           11039.96    40753.91   4523895.79      28589.85       26801.73   
   NaN      14765897.82  3354099.77  31174567.70    1766112.04     1639439.84   
X  유              11.51        6.22      1881.68         20.47          20.39   
   NaN         74118.31     5972.95    122838.98       2691.80        2508.18   

             E_SOx_total  E_NH3_total  E_CO2_total     차대번호    CO   NOx  SOx   
등급 저감장치부착유무                                                                    
3  NaN        1194307.60    118239.56    535449.77  5213000  1.57 10.46 0.23  \
4  유               10.90         0.28         0.50       15  0.04 66.31 0.73   
   NaN         207573.09     13566.04     58273.81   835890  0.99 26.83 0.25   
5  유            49669.36      1241.34      3178.40    72050  0.15 62.79 0.69   
   NaN         315377.72     11202.53     30720.50   673947 21.91 46.26 0.47   
X  유               14.94         1.12        13.06       82  0.14 22.95 0.18   
   NaN           4937.90       404.14      1182.42    19393  3.82  6.33 0.25   

             PM10  PM2.5   HC  NH3  CO2  
등급 저감장치부착유무                              
3  NaN       0.06   0.06 0.22 0.02 0.10  
4  유         0.10   0.09 0.15 0.02 0.03  
   NaN       0.56   0.51 0.33 0.02 0.07  
5  유         0.40   0.37 0.57 0.02 0.04  
   NaN       2.62   2.43 4.98 0.02 0.05  
X  유         0.25   0.25 0.08 0.01 0.16  
   NaN       0.14   0.13 0.31 0.02 0.06

In [144]:
grp2_1 = add.groupby(['등급', '저감장치부착유무', '차종'], dropna=False)['차대번호'].count().unstack('차종')
grp2_1

차종                승용       화물
등급 저감장치부착유무                  
3  NaN       3222270  1990730
4  유               3       12
   NaN        352168   483722
5  유           21827    50223
   NaN        131619   542328
X  유              81        1
   NaN          7277    12116

In [145]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.06.22_PM.05.07.40'

In [146]:
with pd.ExcelWriter(os.path.join(an_fold, '통계_연료_등급별_배출가스배출량.xlsx')) as writer:
    grp1.to_excel(writer, sheet_name='휘발유_가스')
    grp1_1.to_excel(writer, sheet_name='휘발유_가스_차종')
    grp2.to_excel(writer, sheet_name='경유')
    grp2_1.to_excel(writer, sheet_name='경유_차종')

## 배출량 통계2
- 조건
    - 승용(대형)
    - 화물(대형)

In [147]:
df2.loc[(df2['fuel'] == '경유') & (df2['차종'] == '승용') & (df2['차종유형'] == '대형'), '등급'].value_counts(dropna=False)

등급
3    1744356
5     175072
4     151014
X       8308
Name: count, dtype: int64

In [148]:
df2.loc[(df2['fuel'] == '경유') & (df2['차종'] == '화물') & (df2['차종유형'] == '대형'), '등급'].value_counts(dropna=False)

등급
3    189680
5     86825
4     21367
X      1080
Name: count, dtype: int64

In [149]:
ad2 = df2.loc[((df2['차종'] == '승용') & (df2['차종유형'] == '대형')) | ((df2['차종'] == '화물') & (df2['차종유형'] == '대형'))].reset_index(drop=True)
ad2.shape

(6662922, 70)

In [150]:
ad2gl = ad2[(ad2['fuel'] == '휘발유') | (ad2['fuel'] == 'LPG')]
ad2d = ad2[ad2['fuel'] == '경유']
ad2.shape[0] == ad2gl.shape[0] + ad2d.shape[0]

True

In [151]:
grp21 = ad2gl.groupby(['등급'], dropna=False).agg({'E_CO_total':'sum', 'E_HC_total':'sum', 'E_NOx_total':'sum', 'E_PM10_total':'sum', 'E_PM2_5_total':'sum', 'E_SOx_total':'sum', 'E_NH3_total':'sum', 'E_CO2_total':'sum', '차대번호':'count'})
grp21

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차대번호
등급,,,,,,,,,
1,2852082.90,391322.16,267681.55,5656.45,5203.93,12602.31,140918.11,0.00,911275
2,9106732.34,1137239.58,922390.43,18592.55,17105.15,94412.40,487978.87,55063.42,2919269
3,4934266.84,385723.42,883335.95,3890.42,3579.19,10084.80,208550.92,5023.77,384985
4,2173049.49,267507.69,449770.65,432.97,399.00,634.85,38020.38,107.09,36270
5,369398.44,44937.37,25771.51,13.25,12.19,15.46,1169.03,9.36,1108
X,194977.88,24388.37,17420.81,205.32,188.90,556.70,5742.64,84.92,32313


In [152]:
grp21['CO'] = grp21['E_CO_total'] / grp21['차대번호']
grp21['NOx'] = grp21['E_NOx_total'] / grp21['차대번호']
grp21['SOx'] = grp21['E_SOx_total'] / grp21['차대번호']
grp21['PM10'] = grp21['E_PM10_total'] / grp21['차대번호']
grp21['PM2.5'] = grp21['E_PM2_5_total'] / grp21['차대번호']
grp21['HC'] = grp21['E_HC_total'] / grp21['차대번호']
grp21['NH3'] = grp21['E_NH3_total'] / grp21['차대번호']
grp21['CO2'] = grp21['E_CO2_total'] / grp21['차대번호']
grp21

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차대번호,CO,NOx,SOx,PM10,PM2.5,HC,NH3,CO2
등급,,,,,,,,,,,,,,,,,
1,2852082.90,391322.16,267681.55,5656.45,5203.93,12602.31,140918.11,0.00,911275,3.13,0.29,0.01,0.01,0.01,0.43,0.15,0.00
2,9106732.34,1137239.58,922390.43,18592.55,17105.15,94412.40,487978.87,55063.42,2919269,3.12,0.32,0.03,0.01,0.01,0.39,0.17,0.02
3,4934266.84,385723.42,883335.95,3890.42,3579.19,10084.80,208550.92,5023.77,384985,12.82,2.29,0.03,0.01,0.01,1.00,0.54,0.01
4,2173049.49,267507.69,449770.65,432.97,399.00,634.85,38020.38,107.09,36270,59.91,12.40,0.02,0.01,0.01,7.38,1.05,0.00
5,369398.44,44937.37,25771.51,13.25,12.19,15.46,1169.03,9.36,1108,333.39,23.26,0.01,0.01,0.01,40.56,1.06,0.01
X,194977.88,24388.37,17420.81,205.32,188.90,556.70,5742.64,84.92,32313,6.03,0.54,0.02,0.01,0.01,0.75,0.18,0.00


In [153]:
grp21_1 = ad2gl.groupby(['등급', '차종'], dropna=False)['차대번호'].count().unstack('차종')
grp21_1

차종,승용,화물
등급,,
1,911275.00,NaN
2,2919259.00,10.00
3,384981.00,4.00
4,36265.00,5.00
5,1093.00,15.00
X,32289.00,24.00


In [154]:
grp22 = ad2d.groupby(['등급', '저감장치부착유무'], dropna=False).agg({'E_CO_total':'sum', 'E_HC_total':'sum', 'E_NOx_total':'sum', 'E_PM10_total':'sum', 'E_PM2_5_total':'sum', 'E_SOx_total':'sum', 'E_NH3_total':'sum', 'E_CO2_total':'sum', '차대번호':'count'})
grp22

E_CO_total  E_HC_total  E_NOx_total  E_PM10_total  E_PM2_5_total   
등급 저감장치부착유무                                                                     
3  NaN       5491097.22   478204.70  24653074.26     110449.67      101613.92  \
4  유               1.04        2.90       949.28          2.14           1.97   
   NaN        508190.38   100432.37   5793278.03      47406.52       43614.62   
5  유           15858.04    63750.58  10160466.26      87249.40       81107.82   
   NaN       4942502.79  1396594.28  24351985.06    1262250.56     1176417.39   
X  유              11.40        9.66      2278.08         23.41          22.98   
   NaN         35484.36     5997.33    150093.66       4565.20        4253.70   

             E_SOx_total  E_NH3_total  E_CO2_total     차대번호  
등급 저감장치부착유무                                                  
3  NaN         635855.24     74261.71    106457.37  1934036  
4  유                9.99         0.38         2.30       12  
   NaN          62812.89      4707.02     12590.75   172369  
5  유            51858.35      1922.07     12911.87    57050  
   NaN         122264.64      5138.34     29163.17   204847  
X  유               15.02         1.01         1.45       69  
   NaN           3302.86       338.34       552.87     9319

In [155]:
grp22['CO'] = grp22['E_CO_total'] / grp22['차대번호']
grp22['NOx'] = grp22['E_NOx_total'] / grp22['차대번호']
grp22['SOx'] = grp22['E_SOx_total'] / grp22['차대번호']
grp22['PM10'] = grp22['E_PM10_total'] / grp22['차대번호']
grp22['PM2.5'] = grp22['E_PM2_5_total'] / grp22['차대번호']
grp22['HC'] = grp22['E_HC_total'] / grp22['차대번호']
grp22['NH3'] = grp22['E_NH3_total'] / grp22['차대번호']
grp22['CO2'] = grp22['E_CO2_total'] / grp22['차대번호']
grp22

E_CO_total  E_HC_total  E_NOx_total  E_PM10_total  E_PM2_5_total   
등급 저감장치부착유무                                                                     
3  NaN       5491097.22   478204.70  24653074.26     110449.67      101613.92  \
4  유               1.04        2.90       949.28          2.14           1.97   
   NaN        508190.38   100432.37   5793278.03      47406.52       43614.62   
5  유           15858.04    63750.58  10160466.26      87249.40       81107.82   
   NaN       4942502.79  1396594.28  24351985.06    1262250.56     1176417.39   
X  유              11.40        9.66      2278.08         23.41          22.98   
   NaN         35484.36     5997.33    150093.66       4565.20        4253.70   

             E_SOx_total  E_NH3_total  E_CO2_total     차대번호    CO    NOx  SOx   
등급 저감장치부착유무                                                                     
3  NaN         635855.24     74261.71    106457.37  1934036  2.84  12.75 0.33  \
4  유                9.99         0.38         2.30       12  0.09  79.11 0.83   
   NaN          62812.89      4707.02     12590.75   172369  2.95  33.61 0.36   
5  유            51858.35      1922.07     12911.87    57050  0.28 178.10 0.91   
   NaN         122264.64      5138.34     29163.17   204847 24.13 118.88 0.60   
X  유               15.02         1.01         1.45       69  0.17  33.02 0.22   
   NaN           3302.86       338.34       552.87     9319  3.81  16.11 0.35   

             PM10  PM2.5   HC  NH3  CO2  
등급 저감장치부착유무                              
3  NaN       0.06   0.05 0.25 0.04 0.06  
4  유         0.18   0.16 0.24 0.03 0.19  
   NaN       0.28   0.25 0.58 0.03 0.07  
5  유         1.53   1.42 1.12 0.03 0.23  
   NaN       6.16   5.74 6.82 0.03 0.14  
X  유         0.34   0.33 0.14 0.01 0.02  
   NaN       0.49   0.46 0.64 0.04 0.06

In [156]:
grp22_1 = ad2d.groupby(['등급', '저감장치부착유무', '차종'], dropna=False)['차대번호'].count().unstack('차종')
grp22_1

차종                승용      화물
등급 저감장치부착유무                 
3  NaN       1744356  189680
4  유               7       5
   NaN        151007   21362
5  유           30192   26858
   NaN        144880   59967
X  유              66       3
   NaN          8242    1077

In [157]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.06.22_PM.05.08.09'

In [158]:
with pd.ExcelWriter(os.path.join(an_fold, '통계_연료_등급별_배출가스배출량_대형.xlsx')) as writer:
    grp21.to_excel(writer, sheet_name='휘발유_가스')
    grp21_1.to_excel(writer, sheet_name='휘발유_가스_차종')
    grp22.to_excel(writer, sheet_name='경유')
    grp22_1.to_excel(writer, sheet_name='경유_차종')

In [159]:
df2['fuel'].unique()

array(['휘발유', 'LPG', '경유'], dtype=object)

In [160]:
grp3 = df2.groupby('fuel').agg({'E_CO_total':'sum', 'E_HC_total':'sum', 'E_NOx_total':'sum', 'E_PM10_total':'sum', 'E_PM2_5_total':'sum', 'E_SOx_total':'sum', 'E_NH3_total':'sum', 'E_CO2_total':'sum'})
grp3

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total
fuel,,,,,,,,
LPG,13748960.00,467489.48,2524445.82,25962.55,24025.41,365081.14,357617.87,319615.30
경유,48675800.02,9155303.96,219163906.22,4914395.81,4617284.21,3504250.75,279230.97,847346.80
휘발유,113151860.31,11530128.55,11212453.50,93715.34,86282.70,182402.15,2968822.27,569573.74


In [161]:
grp3.to_excel(os.path.join(an_fold, '통계_CAPSS_배출량.xlsx'))

# [출력] 경유, 휘발유, LPG 차량 배출량

In [431]:
df2.columns

Index(['법정동코드', '차종', '차대번호', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN', '차종유형',
       '연료', '등급', 'fuel', '차령', '시도', '시구군', '일일평균주행거리', '저감장치부착유무', 'VKT',
       'DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx', 'R_CO', 'R_HC', 'R_PM', 'EFi_CO',
       'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5', 'E_HOT_CO', 'E_HOT_HC',
       'E_HOT_NOx', 'E_HOT_PM10', 'E_HOT_PM2_5', 'Beta(Estimated)',
       'eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx', 'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM',
       'E_COLD_CO', 'E_COLD_NOx', 'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5',
       'e_d', 'e_RHOT', 'e_SHOT', 'e_SWARM', 'e_fi', 'e_RWARM', 'R', 'S_fi',
       'S_c', 'E_EVA_VOC', 'E_CO_total', 'E_HC_total', 'E_NOx_total',
       'E_PM10_total', 'E_PM2_5_total', 'fcc', 'sf', 'E_SOx_total'],
      dtype='object')

In [432]:
# 컬럼명 정리
ep = df2[[
    '법정동코드', 
    '시도', 
    '시구군', 
    '차대번호', 
    '차량연식', 
    '배출가스인증번호', 
    '등급', 
    'fuel', 
    '차종', 
    '차종유형', 
    '일일평균주행거리', 
    '저감장치부착유무', 
    'E_CO_total', 
    'E_HC_total', 
    'E_NOx_total',
    'E_PM10_total', 
    'E_PM2_5_total', 
    'E_SOx_total',
    ]]
ep.shape

(25117108, 18)

In [434]:
ep.columns

Index(['법정동코드', '시도', '시구군', '차대번호', '차량연식', '배출가스인증번호', '등급', 'fuel', '차종',
       '차종유형', '일일평균주행거리', '저감장치부착유무', 'E_CO_total', 'E_HC_total',
       'E_NOx_total', 'E_PM10_total', 'E_PM2_5_total', 'E_SOx_total'],
      dtype='object')

In [435]:
ep = ep.rename(columns={'등급':'배출가스등급', 'fuel':'연료'})

MemoryError: Unable to allocate 383. MiB for an array with shape (2, 25117108) and data type object

In [ ]:
# # about
# df2.to_csv(os.path.join(df_fold, '경유_휘발유_LPG차량_배출량.csv'), index=False)

# 4등급 차량 차종별 일일평균주행거리 확인
- 이상 주행거리 제외

## [로드] 일일평균주행거리

In [30]:
# about 
name = '일일평균주행거리_최초등록일자_최근검사일자_기준'
file_name = f'{name}.csv'
run = pd.read_csv(os.path.join(raw1_fold, file_name), low_memory=False)
run.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2009566 entries, 0 to 2009565
Data columns (total 8 columns):
 #   Column    Dtype  
---  ------    -----  
 0   차대번호      object 
 1   최초등록일자수정  object 
 2   최근검사일자수정  object 
 3   최근주행거리    int64  
 4   등록일기준검사일  int64  
 5   현재일자      object 
 6   일일평균주행거리  float64
 7   최근검사경과일   int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 122.7+ MB


In [32]:
run.shape, len(run['차대번호'].unique())

((2009566, 8), 2009566)

In [31]:
smig3.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel'],
      dtype='object')

In [34]:
smig3.shape

(25692512, 14)

In [33]:
df = smig3.merge(run[['차대번호', '일일평균주행거리']], on='차대번호', how='left')
df.shape

(25692512, 15)

In [36]:
dfs1 = df[df['일일평균주행거리'] < 1000].reset_index(drop=True)
dfs1.shape

(2009532, 15)

In [38]:
dfs1.groupby('차종').agg({'일일평균주행거리':'mean'}).reset_index()

,차종,일일평균주행거리
0,승용,37.11
1,승합,43.40
2,특수,102.08
3,화물,43.81


In [40]:
dfs1.groupby('차종').agg({'일일평균주행거리':'mean'}).reset_index().to_excel(os.path.join(an1_fold, '통계_차종별_일일평균주행거리.xlsx'), index=False)

In [44]:
df['등급'].unique()

array(['X', '1', '2', '3', '4', '5'], dtype=object)

In [45]:
dfs2 = df[(df['일일평균주행거리'] < 1000) & (df['등급'] == '4')].reset_index(drop=True)
dfs2.shape

(1192850, 15)

In [46]:
dfs2.groupby('차종').agg({'일일평균주행거리':'mean'}).reset_index()

,차종,일일평균주행거리
0,승용,38.29
1,승합,45.29
2,특수,156.36
3,화물,48.21


In [47]:
dfs2.groupby('차종').agg({'일일평균주행거리':'mean'}).reset_index().to_excel(os.path.join(an1_fold, '통계_4등급_차종별_일일평균주행거리.xlsx'), index=False)

# 코드 마지막